In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import pickle
import itertools
import networkx as nx
import scanpy as sc
import matplotlib.pyplot as plt
import pwlf

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/')

from minConf.minConf_PQN import minConF_PQN

# Functions

In [2]:


def ShorestPath_GivenTFs(Gnx, TFlist, FilePathes):
    Ol_TF = list(set(Gnx.nodes) & set(TFlist))
    #print("Only ", len(Ol_TF), " TFs (", len(TFlist), " in total) appear in the network.")
    
    fh = open(FilePathes, 'w')
    ShortPath = list()
    NumNoPath = 0
    for i in range(len(Ol_TF)):
        for j in range(i+1, len(Ol_TF)):
            if nx.has_path(Gnx, Ol_TF[i], Ol_TF[j]):
                sp = nx.shortest_path_length(Gnx, Ol_TF[i], Ol_TF[j])
                Path = nx.shortest_path(Gnx, source=Ol_TF[i], target=Ol_TF[j])
                for n in Path:
                    fh.write("%s " % n)
                fh.write("%d\n" % sp)
                ShortPath.append(sp)
            else:
                fh.write("No path between %s and %s\n" % ( Ol_TF[i], Ol_TF[j]))
                NumNoPath = NumNoPath + 1
                #ShortPath.append(10)
    fh.close()

    return ShortPath

def Short_Path_Matrix(Gnx, TFlist, MaxLen = 6):
    NodesG = Gnx.nodes
    
    N = len(TFlist)
    Smat = np.zeros((N,N))
    
    for i in range(N):
        for j in range(i+1, N):
            if TFlist[i] in NodesG and TFlist[j] in NodesG:
                if nx.has_path(Gnx, TFlist[i], TFlist[j]):
                    Smat[i,j] = nx.shortest_path_length(Gnx, TFlist[i], TFlist[j])
                    Smat[j,i] = Smat[i,j]
                else:
                    Smat[i,j] = MaxLen
                    Smat[j,i] = Smat[i,j]
            else:
                Smat[i,j] = MaxLen
                Smat[j,i] = Smat[i,j]
                
    return Smat


def Short_Path_DSD_Matrix(Gnx, TFlist, MaxLen = 5):
    NodesG = Gnx.nodes
    
    N = len(TFlist)
    Smat = np.zeros((N,N))
    
    for i in range(N):
        for j in range(i+1, N):
            if TFlist[i] in NodesG and TFlist[j] in NodesG:
                Smat[i,j] = Gnx.edges[TFlist[i], TFlist[j]]['weight']
                Smat[j,i] = Smat[i,j]   
                #print(Smat[i,j])
            else:
                #print(TFlist[i], TFlist[j])
                Smat[i,j] = MaxLen
                Smat[j,i] = Smat[i,j]
                
    return Smat



In [3]:
# 

import numpy as np
from scipy.sparse import spdiags
from scipy.linalg import inv
from gurobipy import Model, GRB, QuadExpr

def L0GLObj(u, X, y, pho):
    # u (feature, 1)
    # X (instance, feature)
    # y (instance, 1)
    # pho (1, 1)
    #print(pho)
    #print(u)
    n, d = X.shape
    SpDiag = spdiags(u.flatten(), 0, d, d)
    # print(SpDiag)
    # print(f"SpDiag: {SpDiag.toarray()[-1][-1]}")
    M = inv((1/pho) * X @ SpDiag @ X.conj().T + np.eye(n))
    f = y.conj().T @ M @ y

    g = -(1/pho) * ((X.conj().T @ M @ y)**2)

    return f, g

def L0Obj(u, X, y, pho):
    # u (feature, 1)
    # X (instance, feature)
    # y (instance, 1)
    # pho (1, 1)
    #print(pho)
    n, d = X.shape
    SpDiag = spdiags(u.flatten(), 0, d, d)
    # print(SpDiag)
    # print(f"SpDiag: {SpDiag.toarray()[-1][-1]}")
    M = inv((1/pho) * X @ SpDiag @ X.conj().T + np.eye(n))
    f = y.conj().T @ M @ y

    g = -(1/pho) * ((X.conj().T @ M @ y)**2)

    return f, g

def L0GL_PPI_Obj(u, X, y, S, pho, lam):
    # u (feature, 1)
    # X (instance, feature)
    # y (instance, 1)
    # pho (1, 1)
    #print(pho)
    #print(u)
    n, d = X.shape
    SpDiag = spdiags(u.flatten(), 0, d, d)
    # print(SpDiag)
    # print(f"SpDiag: {SpDiag.toarray()[-1][-1]}")
    M = inv((1/pho) * X @ SpDiag @ X.T + np.eye(n))
    f = y.T @ M @ y + lam * u.T @ S @ u

    g = -(1/pho) * ((X.T @ M @ y)**2) + 2 * lam * S @ u

    return f, g


def ProjCSimplex(u, k):
    """
    Projects vector u onto the simplex defined by the sum of elements equal to k.
    """
    # u = u.flatten()
    # print(u.shape)
    n, m = u.shape
    # print(f"n: {n}, {m}")

    H = 0.5*np.eye(n)
    f = -0.5 * u
    A = np.ones((1, n))
    b = k

    lb = np.zeros((n, 1))
    ub = np.ones((n, 1))
    
    if np.sum(u) < k and np.all(np.logical_and(u >= 0, u <= 1)):
        up = u
    else:
        d = len(u)
        e = np.ones((d, 1))
        z0 = np.zeros((d, 1))
        l0 = float(0 + np.random.rand() * np.min(u))
        error = 100
        idt = 0
        while error > 1e-8:
            tmp = u - l0
            # tmp[tmp > 1] = 1
            # tmp[tmp < 0] = 0
            tmp = np.clip(tmp, 0, 1)
            error = k - np.sum(tmp)
            n = np.count_nonzero(tmp)
            if n == 0:
                l0 = float(0+np.random.rand() * np.min(u))
                error = 100
                continue
            idt += 1
            l1 = l0 - (error / n)
            tmp = u - l1
            # tmp[tmp > 1] = 1
            # tmp[tmp < 0] = 0
            tmp = np.clip(tmp, 0, 1)
            error = np.abs(k - np.sum(tmp))
            l0 = l1
        
        if l0 < 0:
            l0 = 0
        tmp = u - l0
        # tmp[tmp > 1] = 1
        # tmp[tmp < 0] = 0
        tmp = np.clip(tmp, 0, 1)
        up = tmp

    # print(f"up: {up[-1][-1]}")
    
    return up

def ProjCSimplexGL_Gurobi(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A2 = np.zeros((1, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 2)
    b[0] = k
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis]

# used for PQN,comparing to ProjCSimplexGL_Gurobi_LB, it has two output
def ProjCSimplexGL_Gurobi_v2(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A2 = np.zeros((1, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 2)
    b[0] = k
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis], zp[:,np.newaxis]

# used for project, comparing to ProjCSimplexGL_Gurobi_v2, it only has one output
def ProjCSimplexGL_Gurobi_LB(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A1t = -np.ones((1, n))
    A1 = np.vstack([A1, A1t])
    A2 = np.zeros((2, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 3)
    b[0] = k
    b[1] = -k+1
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    #print(x)
    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis]

def ProjCSimplexGL_Gurobi_LB_v2(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A1t = -np.ones((1, n))
    A1 = np.vstack([A1, A1t])
    A2 = np.zeros((2, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 3)
    b[0] = k
    b[1] = -k+1
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis], zp[:,np.newaxis]

In [4]:
def Get_TFs_in_Data(TargetGene, scATAC_df, TF_Col, Gene_GTOpenR_Mapping, Gene_in_Data):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_Raw = Tmp_Input[TF_Col].tolist()
    Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
    TF_Groups_in_Data = list()
    count = 0
    for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
        if chp in Map_Gene_GTOpenR[GL[0]]:
            hit = 1
        else:
            hit = 0
        olp = list(set(tfl) & set(Gene_in_Data))
        TF_Groups_in_Data.append(olp)
        print(chp, count, olp, hit)
        count = count + 1
        
    return TF_Groups_in_Data, Tmp_Input

def Get_TFs_in_Data_noPrint(TargetGene, scATAC_df, TF_Col, Gene_GTOpenR_Mapping, Gene_in_Data):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_Raw = Tmp_Input[TF_Col].tolist()
    Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
    TF_Groups_in_Data = list()
    count = 0
    for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
        if chp in Map_Gene_GTOpenR[GL[0]]:
            hit = 1
        else:
            hit = 0
        olp = list(set(tfl) & set(Gene_in_Data))
        TF_Groups_in_Data.append(olp)
        #print(count, olp, hit)
        count = count + 1
        
    return TF_Groups_in_Data, Tmp_Input



def Show_TFs_in_Data(TargetGene, scATAC_df, TF_Col, Gene_GTOpenR_Mapping, Pred_TFs):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_Raw = Tmp_Input[TF_Col].tolist()
    Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
    TF_Groups_in_Data = list()
    count = 0
    for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
        if chp in Map_Gene_GTOpenR[GL[0]]:
            hit = 1
        else:
            hit = 0
        olp = list(set(tfl) & set(Pred_TFs))
        TF_Groups_in_Data.append(olp)
        print(count, olp, hit)
        count = count + 1
        
    return TF_Groups_in_Data


def Group_Ind(TFinData, TF_Groups_in_Data):
    Map_TF_Ind = {k: v for v, k in enumerate(TFinData)}
    
    GroupT = list()
    for tfl in TF_Groups_in_Data:
        if len(tfl) == 0:
            continue
        tmplist = list()
        for tf in tfl:
            tmplist.append(Map_TF_Ind[tf])
        GroupT.append(tmplist)
        
    return GroupT

def Extract_Solution(u_output, z_output, group_info, k_input, h_input):
    zsort_h = np.argsort(-z_output.flatten())[0:h_input]
    #print("Selected group index:", zsort_h)
    
    Idt_ = list()
    for ht in zsort_h:
        Idt_.extend(group_info[ht])
    Idt_ = np.array(Idt_)
    #print(Idt_)
        
    u_select = u_output.flatten()[Idt_]
    Idt_u = np.argsort(-u_select.flatten())[0:k_input]
    
    #print(u_select[Idt_u])
    
    final_idt = Idt_[Idt_u]
    
    return final_idt

def Split_Sample(X_raw, y_raw, SID, percentage_training):
    NumSample, NumFeature = X_raw.shape
    #print("Number of features:", NumFeature)

    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    #print("Train sample #", len(TrainSampleInd), "Out-of-sample #", len(OutSampleInd))

    X_train = X_raw[TrainSampleInd,:]
    X_outsample = X_raw[OutSampleInd,:] 
    #print(X.shape)
    #print(X_outsample.shape)

    y_train = y_raw[TrainSampleInd]
    y_outsample = y_raw[OutSampleInd]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)

    return X_train, y_train, X_outsample, y_outsample, SID_train, SID_outsample

def Split_Sample_scRNA(scRNA_Anndata, percentage_training):
    NumSample, NumFeature = scRNA_Anndata.shape
    SID = scRNA_Anndata.obs.index.tolist()
    
    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)
    
    scRNA_Anndata_Train = scRNA_Anndata[TrainSampleInd,:]
    scRNA_Anndata_OutSample = scRNA_Anndata[OutSampleInd,:] 
    
    return scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample
    



In [5]:
def DF_Column_to_List(DataFrame, ColName):
    return list(set(list(itertools.chain.from_iterable(DataFrame[ColName].tolist()))))

def Split_Sample_scRNA(scRNA_Anndata, percentage_training):
    NumSample, NumFeature = scRNA_Anndata.shape
    SID = scRNA_Anndata.obs.index.tolist()
    
    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)
    
    scRNA_Anndata_Train = scRNA_Anndata[TrainSampleInd,:]
    scRNA_Anndata_OutSample = scRNA_Anndata[OutSampleInd,:] 
    
    return scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample

def Get_TFs_with_Exp(TargetGene, scATAC_df, TF_Col, Gene_in_scRNA):
    GL = [TargetGene]
    
    # get df for the target gene
    Tmp_Input = scATAC_df[scATAC_df['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    TFList_All = list(set(itertools.chain.from_iterable(Tmp_Input[TF_Col].tolist())))
    
    TFList_with_Exp = list(set(TFList_All) & set(Gene_in_scRNA))
        
    return TFList_with_Exp

def Short_Path_DSD_Matrix(Gnx, TFlist, MaxLen = 5):
    NodesG = Gnx.nodes
    
    N = len(TFlist)
    Smat = np.zeros((N,N))
    
    for i in range(N):
        for j in range(i+1, N):
            if TFlist[i] in NodesG and TFlist[j] in NodesG:
                Smat[i,j] = Gnx.edges[TFlist[i], TFlist[j]]['weight']
                Smat[j,i] = Smat[i,j]   
                #print(Smat[i,j])
            else:
                #print(TFlist[i], TFlist[j])
                Smat[i,j] = MaxLen
                Smat[j,i] = Smat[i,j]
                
    return Smat

def Out_of_Sample_MSE(scRNA, TG, TFList, SID_train, SID_outsample, pho):
    XSelect = scRNA[SID_train, TFList].X
    y = scRNA[SID_train, TG].X
    w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(TFList))) @ XSelect.T @ y
    #print(TF_Select)
    #print(np.abs(w_est.flatten()))

    X_outsample_select = scRNA[SID_outsample, TFList].X
    y_outsample = scRNA[SID_outsample, TG].X
    MSE = (1. / len(SID_outsample)) * np.linalg.norm(y_outsample - X_outsample_select @ w_est)
    
    return MSE

In [6]:
def ProjCSimplexGL_Gurobi_LB_v2(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A1t = -np.ones((1, n))
    A1 = np.vstack([A1, A1t])
    A2 = np.zeros((2, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 3)
    b[0] = k
    b[1] = -k+1
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis], zp[:,np.newaxis]

def Extract_Solution(u_output, z_output, group_info, k_input, h_input):
    zsort_h = np.argsort(-z_output.flatten())[0:h_input]
    #print("Selected group index:", zsort_h)
    
    Idt_ = list()
    for ht in zsort_h:
        Idt_.extend(group_info[ht])
    Idt_ = np.array(Idt_)
    #print(Idt_)
        
    u_select = u_output.flatten()[Idt_]
    Idt_u = np.argsort(-u_select.flatten())[0:k_input]
    
    #print(u_select[Idt_u])
    
    final_idt = Idt_[Idt_u]
    
    return final_idt


def Split_Sample_scRNA(scRNA_Anndata, percentage_training):
    NumSample, NumFeature = scRNA_Anndata.shape
    SID = scRNA_Anndata.obs.index.tolist()
    
    RandInd = np.random.permutation(NumSample)
    NumTrainSample = int(percentage_training*NumSample)
    TrainSampleInd = RandInd[0:NumTrainSample]
    OutSampleInd = RandInd[NumTrainSample:]
    
    SID_train = list( SID[i] for i in TrainSampleInd)
    SID_outsample = list( SID[i] for i in OutSampleInd)
    
    scRNA_Anndata_Train = scRNA_Anndata[TrainSampleInd,:]
    scRNA_Anndata_OutSample = scRNA_Anndata[OutSampleInd,:] 
    
    return scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample

def Find_best_k(KRange, MES_Array):
    
    best_k = -1
    
    if (MES_Array[0] - MES_Array[-1]) / MES_Array[0] <= 0.3:
        return best_k
    else:
        my_pwlf = pwlf.PiecewiseLinFit(KRange, MES_Array)
        breaks = my_pwlf.fit(2)
        best_k = np.ceil(breaks[1])
        return int(best_k)
        

def Run_GRIPNet(scRNA_Anndata, SID_train, SID_outsample, scATAC_DF, DisMat_PPI, TF_Motif_Metric, TargetGenes, TFNum_Range = range(3,11), Lambda = 0.0001, Pho = 0.0001): 
    
    Genes_with_Exp = scRNA_Anndata[SID_train,:].var.index.tolist()
    
    GRIPNet_DF = pd.DataFrame(columns=['TG', 'TFs', 'Out_of_Sample MSE'])
    count = 1
    for tg in TargetGenes:
        if tg not in Genes_with_Exp:
            print("No expression data for ", tg)
            continue
            
        # Collect all TFs (with gene expression), whose motifs matched to the open regions within 250kb around TG and 
        TFs_for_TG = Get_TFs_with_Exp(tg, scATAC_DF, TF_Motif_Metric, Genes_with_Exp)
        #print(TFs_for_TG)
        
        if len(TFs_for_TG) <= 3:
            count = count + 1
            continue
        
        # Parameters setup for the algo.
        NumFeature = len(TFs_for_TG)
        NumSample = scRNA_Anndata[:,TFs_for_TG].X.shape[0]
        
        # Regression data
        X = scRNA_Anndata[SID_train,TFs_for_TG].X
        y = scRNA_Anndata[SID_train,tg].X
        uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
        
        # Distance between poential TFs
        S =  Short_Path_DSD_Matrix(DisMat_PPI, TFs_for_TG)
        #print(S.shape)
      
        Group = [range(NumFeature)]
        h = 1
        lamb = Lambda
        pho = Pho

        if len(TFs_for_TG) < TFNum_Range[-1]:
            K_Range = range(3, len(TFs_for_TG)+1)
        else:
            K_Range = TFNum_Range
        MSE_ = np.zeros(len(K_Range))
        TF_Select_Collect = {}
        Map_Ind_k = {}
        for i in range(len(K_Range)):
            k = K_Range[i]
    
            #print(lamb)
            # Set up Objective Function
            funObj_PPI = lambda w: L0GL_PPI_Obj(w, X, y, S, pho, lamb)
            # Set up Simplex Projection Function
            funProj_GL_LB = lambda w: ProjCSimplexGL_Gurobi_LB(w, k, Group, h)
            # parameters that can be changed
            options = {'maxIter': 400, 'verbose': -1}
            
            uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
            uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
            SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)

            TF_Select = list()
            for idt in SelectID:
                TF_Select.append(TFs_for_TG[idt])
            #print(TF_Select)
                
            TF_Select_Collect[k] = TF_Select


            MSE_[i] = Out_of_Sample_MSE(scRNA_Anndata, tg, TF_Select, SID_train, SID_outsample, pho)
            Map_Ind_k[i] = k
            #print(k, MSE_[i])
            
        # find the best k    
        best_k = Map_Ind_k[np.argmin(MSE_)]#Find_best_k(K_Range, MSE_)
        if best_k == -1:
            print("Fail to have good inference for ", tg)
        else:
            print(count, '/', len(TargetGenes), ":", tg, best_k, TF_Select_Collect[best_k], MSE_[K_Range.index(best_k)])
            GRIPNet_DF = GRIPNet_DF._append({'TG': tg, 'TFs': TF_Select_Collect[best_k], 'Out_of_Sample MSE': MSE_[K_Range.index(best_k)]}, ignore_index=True)
        count = count + 1
    return GRIPNet_DF
        
        

In [7]:
MSE = np.array([0.3, 0.2, 0.15, 0.4])

np.argmin(MSE)

2

# 1. Load data

## 1.1 Load scRNA-seq and trim to CD4

In [8]:
# load the data
Pbmc_3k_scRNA_DN_magic_ad = ad.read_h5ad("../../Data/PBMC_3k/pmbc_3k_scRNAseq_magic.h5ad")

In [9]:
# trim to CD4 
CD8_scRNA_DN_magic_ad = Pbmc_3k_scRNA_DN_magic_ad[Pbmc_3k_scRNA_DN_magic_ad.obs.ingest_celltype_label == 'CD8 T cells'] 

In [10]:
# further filtering
CD8_scRNA_DN_magic_ad = CD8_scRNA_DN_magic_ad[:, CD8_scRNA_DN_magic_ad.var['mean']>=0.1]

#CD4_scRNA_DN_magic_ad = CD4_scRNA_DN_magic_ad[:, CD4_scRNA_DN_magic_ad.var['std']<=0.5]

In [11]:
Gene_in_CD8_scRNA = list(CD8_scRNA_DN_magic_ad.var.index)
Cell_in_CD8_scRNA = list(CD8_scRNA_DN_magic_ad.obs.index)

print(len(Gene_in_CD8_scRNA))
print(len(Cell_in_CD8_scRNA))

3055
195


## 1.2 Load scATAC-seq data and trim to CD8

In [12]:
pbmc_3k_scATAC_Gene_Enhancer = pd.read_pickle('../../Data/PBMC_3k/Pbmc_3k_scATAC_Gene_TF_Enhancer_fimo_multiple_cutoffs.pkl')

In [13]:
# Trim to Cells in CD8
Col_select = ['chr_position', 'Overlapped_enhancer',
 'Genes_within_250k',
 'TF_Matching_p-value_1e-06',
 'TF_Matching_p-value_1e-05',
 'TF_Matching_p-value_0.0001',
 'TF_Matching_q-value_0.01',
 'TF_Matching_q-value_0.05'] + Cell_in_CD8_scRNA
pbmc_3k_scATAC_Gene_Enhancer_CD8 = pbmc_3k_scATAC_Gene_Enhancer[Col_select]

In [14]:
# sum the peaks for cells in CD4
pbmc_3k_scATAC_Gene_Enhancer_CD8['sum'] = pbmc_3k_scATAC_Gene_Enhancer_CD8[Cell_in_CD8_scRNA].values.astype(int).sum(axis=1)

pbmc_3k_scATAC_Gene_Enhancer_CD8 = pbmc_3k_scATAC_Gene_Enhancer_CD8.drop(columns=Cell_in_CD8_scRNA)

/var/folders/x5/p97rd26d6y719tstlhqtn9zr0000gr/T/ipykernel_71972/2372763761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbmc_3k_scATAC_Gene_Enhancer_CD8['sum'] = pbmc_3k_scATAC_Gene_Enhancer_CD8[Cell_in_CD8_scRNA].values.astype(int).sum(axis=1)


In [15]:
# consider only the open region with peaks >=100
peak_sum_threshold = 20
pbmc_3k_scATAC_Gene_Enhancer_CD8_psum = pbmc_3k_scATAC_Gene_Enhancer_CD8[pbmc_3k_scATAC_Gene_Enhancer_CD8['sum']>=peak_sum_threshold]

In [16]:
pbmc_3k_scATAC_Gene_Enhancer_CD8_psum

,chr_position,Overlapped_enhancer,Genes_within_250k,TF_Matching_p-value_1e-06,TF_Matching_p-value_1e-05,TF_Matching_p-value_0.0001,TF_Matching_q-value_0.01,TF_Matching_q-value_0.05,sum
1,chr1_180582_181297,[970:11.4995458702954:CD34+],"[DDX11L1, DDX11L17, FAM138A, LOC112268260, LOC...","[KLF3, NRF1, RREB1, THA]","[BRCA1, COT1, COT2, CTCF, HEN1, KLF1, KLF12, K...","[AP2D, BHE41, BRCA1, COT1, COT2, CTCF, CTCFL, ...","[KLF3, NRF1]","[KLF1, KLF12, KLF3, MAZ, NRF1, RREB1, SP1, SP2...",23
9,chr1_633556_634476,"[550:12.2065412268318:SHEP-21N,787:6.466380502...","[FAM41C, FAM87B, LINC00115, LINC01128, LINC014...",[NR2C1],"[GLI3, GSX2, HXB13, HXC9, INSM1, NR2C1, NRF1, ...","[BCL6, BMAL1, BSH, CDX2, CEBPZ, COT2, CUX1, DL...",[.],"[SP2, ZFX]",86
10,chr1_778284_779202,"[1980:8.91789239746368:CD8+,370:15.50080805540...","[AGRN, FAM41C, FAM87B, HES4, ISG15, KLHL17, LI...","[CEBPZ, FOXI1, NFYB, TEAD1, ZN563, ZN770]","[AP2D, ATF1, ATF2, ATF7, BHA15, CEBPZ, COE1, C...","[AIRE, AP2A, AP2B, AP2C, AP2D, ARNT, ASCL1, AS...",[ZN770],"[AP2D, E2F4, E2F6, EGR1, EGR2, KLF1, PROX1, PU...",268
13,chr1_827063_827952,"[10140:9.36438558234664:HepG2,35010:12.1274907...","[AGRN, FAM41C, FAM87B, HES4, ISG15, KLHL17, LI...","[DPRX, E2F4, EGR1, EGR2, KLF12, KLF15, KLF3, K...","[AP2C, AP2D, DPRX, E2F1, E2F3, E2F4, E2F6, E2F...","[AP2A, AP2B, AP2C, AP2D, ARNT, ARNT2, ASCL2, A...","[EGR1, EGR2, KLF12, KLF15, KLF3, KLF6, PATZ1, ...","[AP2D, E2F4, E2F6, EGR1, EGR2, EGR4, FLI1, KLF...",106
17,chr1_869472_870376,"[4690:17.0572353953428:HEK293T,4690:7.53789980...","[AGRN, C1orf159, FAM41C, FAM87B, HES4, ISG15, ...","[ASCL1, CTCF, CTCFL, EGR1, EGR2, ESR2, KLF14, ...","[AP2B, AP2C, AP2D, ARI3A, ASCL1, BRCA1, COT1, ...","[ANDR, AP2A, AP2B, AP2C, AP2D, ARI3A, ARNT, AR...","[CTCF, CTCFL, EGR1, EGR2, KLF3, KLF9, SP1, SP3...","[AP2D, ASCL1, CTCF, CTCFL, EGR1, EGR2, EGR4, K...",34
...,...,...,...,...,...,...,...,...,...
98304,chrUn_GL000219v1_44650_45512,[.],"[LOC105379420, LOC283788]","[BC11A, ETV5, FLI1, IRF3, MAZ, P63, P73, PATZ1...","[BC11A, E2F7, EGR1, EGR2, EGR4, ELF5, ETS2, ET...","[ANDR, ARNT2, BARX1, BC11A, BHA15, COE1, E2F1,...","[FLI1, IRF3, KLF16, MAZ, PATZ1, SP1, VEZF1, WT...","[BC11A, EGR1, EGR2, EGR4, ETS2, ETV5, FLI1, IR...",29
98305,chrUn_GL000219v1_45733_46550,[.],"[LOC105379420, LOC283788]",[HIC2],"[ARNT2, CEBPD, E2F6, EGR2, HIC2, HNF1A, HNF1B,...","[ANDR, AP2A, AP2C, AP2D, ARNT2, ATF6A, ATOH1, ...",[ZN263],"[EGR2, EGR4, KLF1, KLF3, KLF5, KLF6, PATZ1, PI...",61
98307,chrUn_GL000219v1_90065_90960,[.],"[LOC105379420, LOC283788]","[CDX2, ETS1, TF7L1, TF7L2, ZNF18]","[AIRE, ARI3A, BRAC, CDX2, DUX4, ETS1, ETV3, FO...","[AIRE, ALX1, ARI3A, ATF6A, BARX1, BARX2, BATF3...",[.],"[CPEB1, IRF3, RREB1, STAT1]",20
98309,chrUn_GL000219v1_99257_100164,[.],"[LOC105379420, LOC283788]","[E2F1, E2F4, E2F6, E2F7, ELF1, ELF2, ELK1, ELK...","[AP2D, ARNT2, BHA15, E2F1, E2F3, E2F4, E2F6, E...","[ANDR, AP2A, AP2B, AP2C, AP2D, ARNT2, ASCL1, A...","[SP3, TBX15, WT1, ZN467, ZN770]","[AP2D, E2F4, E2F6, E2F7, EGR1, EGR2, EGR4, KLF...",406


In [17]:
TFList_qv0d01 = DF_Column_to_List(pbmc_3k_scATAC_Gene_Enhancer_CD8_psum, 'TF_Matching_q-value_0.01')
print(len(TFList_qv0d01))

TFList_qv0d05 = DF_Column_to_List(pbmc_3k_scATAC_Gene_Enhancer_CD8_psum, 'TF_Matching_q-value_0.05')
print(len(TFList_qv0d05))

TFList_pv1en6 = DF_Column_to_List(pbmc_3k_scATAC_Gene_Enhancer_CD8_psum, 'TF_Matching_p-value_1e-06')
print(len(TFList_pv1en6))


101
180
591


In [18]:
Gene_in_CD8_scATAC = DF_Column_to_List(pbmc_3k_scATAC_Gene_Enhancer_CD8_psum, 'Genes_within_250k')
print(len(Gene_in_CD8_scATAC))

Gene_in_CD8_scATAC_scRNA = list(set(Gene_in_CD8_scATAC) & set(Gene_in_CD8_scRNA))
print(len(Gene_in_CD8_scATAC_scRNA))

36156
2814


## 1.3 Load PPIs

In [19]:
G_PPI_BioGrid = pickle.load(open('../../Data/PPI/PPI_BioGrid_Evd2_DSD.pickle', 'rb'))
print("The PPI netowrk has", len(G_PPI_BioGrid.nodes), "nodes and", len(G_PPI_BioGrid.edges), "edges")

The PPI netowrk has 14433 nodes and 104148528 edges


# 2 GRIPNet

In [20]:
TF_Motif_Metric = ['TF_Matching_p-value_1e-06',
 'TF_Matching_p-value_1e-05',
 'TF_Matching_p-value_0.0001',
 'TF_Matching_q-value_0.01',
 'TF_Matching_q-value_0.05']

# split data into training and out-of-sample
scRNA_Anndata_Train, scRNA_Anndata_OutSample, SID_train, SID_outsample = Split_Sample_scRNA(CD8_scRNA_DN_magic_ad, percentage_training=0.9)

with open('./CD8_Train_Samples.txt', 'w') as f:
    for sid in SID_train:
        f.write(f"{sid}\n")
        
with open('./CD8_Out_Samples.txt', 'w') as f:
    for sid in SID_outsample:
        f.write(f"{sid}\n")


In [21]:
TGList = ['HIVEP1', 'CYBB', 'ANKRD26', 'IL32', 'ZC4H2', 'PLXNC1', 'AAMDC', 'HLA-DRB1', 'SFXN1', 'EIF2B3', 'CCNC', 'ENTPD1', 'ITGAX', 'LENG1', 'RPL11', 'HIBADH', 'NHSL1', 'ETV6', 'ELP4', 'LIMK2', 'KDM1B', 'AMZ1', 'ADAM9', 'GPR137B', 'MPZL3', 'TUBGCP5', 'MYO9A', 'FCGR2A', 'LDLRAP1', 'ARSG', 'ZNF544', 'PELI2', 'GFPT1', 'SLC35E2B', 'ANXA1', 'NEPRO', 'DUSP6', 'ATP8B4', 'ATP10A', 'MARK4', 'WARS2-AS1', 'GRAMD1B', 'RABEP2', 'NUP54', 'MAML2', 'SNX9', 'HLA-DPB1', 'TNFSF10', 'RHOT1', 'DIPK1A', 'MBNL3', 'RPS29', 'STAT4', 'ATF7IP2', 'RFX7', 'CHSY1', 'LEPROTL1', 'LTB', 'RPS5', 'ZSWIM6', 'BATF', 'JMJD1C', 'CC2D2A', 'ZNF487', 'PSIP1', 'SMAD3', 'PLD1', 'ZDHHC20', 'SOCS3', 'DENND2D', 'ZNF236', 'ATG13', 'LCLAT1', 'RPSA', 'FAM198B-AS1', 'CMC1', 'POU2F2', 'P4HA1', 'LIME1', 'LRRC8D', 'GZMH', 'AIF1', 'BTD', 'FAM153CP', 'PACSIN2', 'RPL18A', 'NR3C2', 'RAE1', 'PLCG1', 'ATP1B3', 'CALR', 'ZC2HC1A', 'FNTB', 'TMOD2', 'FOS', 'SNX30', 'DOCK9', 'RAB11FIP4', 'ATXN1', 'UXS1', 'LINC01237', 'TYMP', 'UGCG', 'STK25', 'HIST1H1C', 'MINDY2', 'FILIP1L', 'SYNJ2', 'ACACA', 'LANCL1', 'P2RX7', 'CST7', 'OPTN', 'SLC7A6', 'TNFRSF25', 'SLC44A1', 'SPPL2A', 'TFEC', 'ATF3', 'HCST', 'UBASH3B', 'CCND3', 'RPL3', 'SCML4', 'GPR183', 'RGS10', 'IL1B', 'COMMD10', 'PYM1', 'DDB2', 'RIC8B', 'FGL2', 'SHQ1', 'PTK2', 'TUT4', 'MS4A7', 'OXNAD1', 'ATP6AP2', 'CHKA', 'ARHGAP32', 'TMTC2', 'HDAC3', 'HLA-DMB', 'MGST2', 'CD82', 'MAML3', 'ZNF782', 'CD2AP', 'GNG7', 'CTNNBIP1', 'PDE4B', 'PCGF5', 'BCAS2', 'SMYD3', 'CHMP7', 'MXD1', 'NR4A3', 'MPRIP', 'ZNF804A', 'DGLUCY', 'MAP3K8', 'ING3', 'SKAP2', 'SLC31A1', 'FGR', 'OAS1', 'CCPG1', 'PRKCE', 'CBR4', 'SLMAP', 'LINC00513', 'RPL27A', 'ZNF81', 'CEP250', 'NPC1', 'SMIM14', 'LINC00926', 'RBM47', 'SMURF1', 'ALDH2', 'SLC25A16', 'LINC00894', 'UTRN', 'MAFB', 'CYB561A3', 'ZNF710', 'TRIO', 'RMND5B', 'TAPT1', 'CYB5D2', 'ARHGAP18', 'ERCC8', 'FHIT', 'KLHL8', 'IQCB1', 'CYSLTR1', 'SLC24A4', 'PIGB', 'CD79A', 'KANTR', 'WDR60', 'HS2ST1', 'TREM1', 'FAM168A', 'CD8A', 'KLF4', 'FCN1', 'ANK3', 'ARHGAP10', 'CTSC', 'RPL14', 'TVP23C', 'LEF1-AS1', 'CCL5', 'BANK1', 'SYNJ1', 'RPS3A', 'TEX14', 'SOD2', 'BSDC1', 'KIF16B', 'GPD2', 'GNB4', 'PHACTR4', 'SPATA1', 'PHACTR1', 'RPS27', 'MANBA', 'EXT2', 'PIGX', 'IL3RA', 'PLAAT4', 'LAIR2', 'ZNF438', 'IFT88', 'ZFAT', 'TTC27', 'ANO10', 'ACAD11', 'FAM222B', 'SLC49A4', 'LIPE-AS1', 'AOAH', 'HSPB1', 'CCDC146', 'WDR27', 'ZDHHC14', 'HIF1A', 'FGFBP2', 'RNF144B', 'EXOC6B', 'TAMM41', 'FGD4', 'C2CD3', 'BCL11A', 'GCNT1', 'SLC8A1', 'TBC1D4', 'VPS54', 'FCRL1', 'SHLD2', 'HLCS', 'ARHGAP31', 'STX11', 'INPP5A', 'CSGALNACT1', 'TASP1', 'ITPRIP', 'TMEM135', 'DHTKD1', 'S100A8', 'ZNF169', 'NF1', 'CDK19', 'ARFGAP3', 'VRK2', 'GPATCH11', 'RXRA', 'PTGER2', 'UBE2W', 'FRMD4B', 'MAN1C1', 'FAM241A', 'IFIH1', 'IFI44', 'NFATC2', 'EAF2', 'GALM', 'ARSB', 'HERC3', 'PHTF2', 'CASK', 'DST', 'JDP2', 'DEPDC5', 'LCORL', 'SETBP1', 'XRCC4', 'GNB5', 'RPS20', 'UBE2D1', 'PICALM', 'CHD7', 'UBTF', 'ARHGEF3', 'ABCB10', 'SUPT3H', 'KIAA1958', 'S100A6', 'WDR6', 'RALGPS2', 'LANCL2', 'TCP11L1', 'JUN', 'USP28', 'GCFC2', 'HAVCR2', 'MCTP2', 'DGKG', 'BMPR2', 'FMN1', 'COLGALT1', 'DNAJC1', 'CPQ', 'CCND2', 'HLA-DPA1', 'PIM2', 'BLK', 'XBP1', 'IRF8', 'CD28', 'OCIAD2', 'TAFA1', 'ARRB1', 'FRY', 'ACTG1', 'S100A10', 'TMC8', 'SRRT', 'PLCL1', 'LLGL2', 'ITFG1', 'MAL', 'CHST11', 'SHLD1', 'OSBPL1A', 'LINC01934', 'CXCR4', 'TCAIM', 'ETS1', 'SPIN1', 'STAT5B', 'TNFAIP2', 'ZDHHC13', 'RPLP1', 'RPL13A', 'BICD1', 'EXT1', 'KRCC1', 'RNF125', 'LINC00861', 'CD8B', 'LPCAT2', 'SEMA4B', 'CSF3R', 'C12orf42', 'WARS2', 'RNF149', 'UBE2J1', 'PLA2G4A', 'PLSCR1', 'TSHZ3', 'ZFHX3', 'SATB1', 'CROCC', 'STMN1', 'CIAPIN1', 'ZFP90', 'ARAF', 'TDP2', 'PCCA', 'ITPR1', 'PRAM1', 'CASC3', 'EPHA4', 'SWAP70', 'NDUFAF2', 'SAMD3', 'FRMD3', 'PPP1R12B', 'RPS12', 'IDE', 'PATJ', 'ZNF512', 'LAP3', 'ZNF10', 'ORAI2', 'PIP5K1B', 'TRMO', 'NT5C2', 'IL2RB', 'TIGAR', 'SULF2', 'ANKRD28', 'ZNF821', 'THAP11', 'ATG4C', 'EBF1', 'RRBP1', 'MS4A1', 'CYTOR', 'DENND1A', 'FAM135A', 'GRK5', 'IFI30', 'MSI2', 'DUSP5', 'E2F3', 'PMAIP1', 'FTL', 'PTPRE', 'NOTCH2', 'BMT2', 'CDK6', 'CUX1', 'LINC02328', 'FAM102B', 'SNX2', 'OXSR1', 'SLC35E3', 'CD226', 'USP36', 'CD83', 'LATS2', 'EEF1B2', 'KHDRBS2', 'SLC16A10', 'RNF135', 'RPS6KA2', 'FOXO1', 'GPHN', 'ZEB2', 'MPP7', 'SPTBN1', 'LINC02432', 'SCO2', 'CLEC7A', 'CCDC66', 'CPD', 'TET2', 'MYBL1', 'DLG2', 'ATP11A', 'JAK2', 'FOSB', 'DIAPH2', 'TTYH3', 'USP32', 'DYSF', 'SPI1', 'WWOX', 'GFM2', 'SLC9A7', 'GANC', 'LIN52', 'BCAS4', 'TMEM176B', 'KCNQ1', 'NOLC1', 'ANKZF1', 'LINC01278', 'SNAPC5', 'FOXO3', 'GBF1', 'SVIL', 'TBC1D14', 'SLC2A4RG', 'TXK', 'ATRN', 'CD14', 'VTA1', 'LST1', 'USP53', 'VNN2', 'DIPK2A', 'UBE2D2', 'GNL2', 'SCML1', 'IL13RA1', 'LINC01572', 'TTN', 'SLC2A3', 'SCAI', 'ANKIB1', 'NLRC3', 'ODF2L', 'KIF20B', 'HCK', 'ADGRE2', 'PRDM1', 'NIBAN3', 'DGKA', 'TGFBR3', 'CLYBL', 'YBX3', 'CEP290', 'PDE4D', 'PTPN4', 'WDFY1', 'CTC1', 'ARHGAP5', 'DTNB', 'AKAP10', 'DENND11', 'ANPEP', 'FBXO32', 'RCAN3', 'MIR181A1HG', 'RAB6A', 'LIX1L', 'DSTYK', 'CARD11', 'PTPN12', 'HNRNPLL', 'STAMBPL1', 'EXOC6', 'C12orf57', 'ABHD5', 'TBC1D8', 'SNX29', 'PITPNC1', 'EPHX2', 'ZEB1', 'MANF', 'ZNF516', 'RETREG1', 'MAP3K5', 'SATB1-AS1', 'CD22', 'BBS9', 'NFKBIA', 'OASL', 'VCL', 'FAM122C', 'PLEKHM3', 'SERINC5', 'SBF2', 'ARRDC3', 'PLEK', 'RPGR', 'LAT', 'OSBPL3', 'ARID5B', 'GPAT3', 'SLC39A11', 'CHST15', 'TRAF3IP2-AS1', 'KIF1B', 'KDM7A', 'CABIN1', 'MSRA', 'AFF3', 'MS4A6A', 'CDC42EP3', 'PRR5', 'KIFAP3', 'SNX25', 'GCH1', 'RPL13', 'TBXAS1', 'MTG1', 'RPS14', 'OLA1', 'FAM13A', 'DUSP1', 'MCPH1', 'ANKUB1', 'BCL2', 'ANTXR2', 'FKBP11', 'ACADM', 'TPCN1', 'JAZF1', 'ITGA6', 'PRMT3', 'ZNF814', 'ADAP2', 'BACH1', 'CENPK', 'RPL38', 'QSOX1', 'ERGIC1', 'DUSP16', 'PCNX4', 'CLEC4E', 'IGF1R', 'NEAT1', 'QKI', 'GALK2', 'NAA10', 'SUSD3', 'MYADM', 'OXCT1', 'GRK3', 'LARS2', 'RPL34', 'RAB28', 'NEDD9', 'SERPINA1', 'GZMA', 'MIS18BP1', 'ERLEC1', 'TMEM263', 'CARS2', 'GPATCH2', 'USF3', 'TRPM2', 'AUTS2', 'CCR2', 'FAM49A', 'PTPN22', 'CHPT1', 'ADCY9', 'RNF168', 'SLC16A7', 'BTRC', 'GAS7', 'ABCG1', 'CHRM3-AS2', 'TRAF3', 'PDE6D', 'ZNF490', 'SGPP1', 'INTS4', 'ITSN1', 'YJU2', 'MAGED1', 'PDK3', 'CYSTM1', 'LRIG1', 'HIST1H1D', 'SLCO3A1', 'HCFC2', 'GIMAP5', 'NOSIP', 'NCALD', 'EVA1C', 'DHX57', 'KLRD1', 'TIPARP', 'FARP2', 'SIPA1L1', 'CDC25B', 'HERC6', 'ANKAR', 'IRF1', 'ZBTB8OS', 'PSMA1', 'GTDC1', 'SPOCK2', 'SCFD2', 'RPL30', 'RELB', 'PSAP', 'MOSMO', 'LINC01184', 'CDKN1C', 'ACSL1', 'RAB3IP', 'EFCAB2', 'WDFY4', 'CTSD', 'ABCA7', 'NLK', 'SYK', 'CBLB', 'NARS2', 'RECK', 'CSF1R', 'ST3GAL3', 'AGTPBP1', 'RPS4X', 'IFI6', 'HLA-DMA', 'ANKH', 'ARHGAP15', 'EMSY', 'LYPLAL1', 'MGAT4A', 'TERF2IP', 'GBP1', 'PAX5', 'CCDC7', 'PVT1', 'CAAP1', 'PPARGC1B', 'RAP1A', 'PPFIBP2', 'THEM4', 'HK2', 'EPS8', 'TMEM117', 'TCF7L2', 'GALT', 'TMEM267', 'PIK3AP1', 'TRABD2A', 'TANGO6', 'S100A12', 'TESK2', 'PDLIM5', 'RPS23', 'SNTB2', 'UBN1', 'GK5', 'MSH3', 'RYBP', 'SOX4', 'MAP4', 'AKR1B1', 'MARCH3', 'TMEM154', 'RAPGEF6', 'IL17RA', 'TLR4', 'NGRN', 'CNNM2', 'DGKH', 'SPATA5', 'WDR25', 'HIST1H1E', 'SIDT1', 'TNFRSF10B', 'NCK2', 'STYXL1', 'NFAM1', 'CTSS', 'SNAP29', 'BAZ2B', 'GK', 'DOCK5', 'BACH2', 'ABL2', 'TXNDC16', 'SH3RF1', 'IFT57', 'COL19A1', 'DLEU1', 'PIGU', 'MMS22L', 'SMC6', 'OPA3', 'ZAP70', 'GTPBP6', 'EPB41L2', 'SLC22A15', 'CR1', 'USP6NL', 'GAB1', 'TBC1D2', 'C5AR1', 'IL15', 'RRM1', 'POLR1C', 'PEAK1', 'FRS2', 'SRGAP2', 'BABAM2', 'SYAP1', 'PCED1B', 'BASP1', 'MX1', 'CCR7', 'SEC22A', 'GMDS-DT', 'EP400', 'PECAM1', 'DENND1C', 'MRPS26', 'CEBPB', 'ACYP2', 'ECHS1', 'PLCB1', 'PLXDC2', 'DUSP2', 'STRBP', 'CBX5', 'NABP1', 'CLEC12A', 'NIBAN1', 'SLC38A1', 'TIMM8B', 'ZC3H6', 'RPL9', 'ESR2', 'DENND5A', 'NBEA', 'SLC25A37', 'ZNF565', 'PDE3B', 'IFITM3', 'UVSSA', 'TECR', 'HSDL2', 'ADAM19', 'PRR5L', 'ULK4', 'KYNU', 'CD5', 'INKA2', 'HSPA1A', 'RAB31', 'GIT2', 'ACSS2', 'CLOCK', 'PLEKHG2', 'DHX35', 'RBM41', 'PCNA', 'PPP1R3E', 'GMEB1', 'SLC11A1', 'TFDP2', 'DISC1', 'LBH', 'NSMAF', 'LILRB1', 'ACER3', 'CARMIL1', 'CKMT2-AS1', 'PPP2R2B', 'DPP4', 'SETD7', 'STK17A', 'PRIM2', 'GZMB', 'AHI1', 'TBC1D19', 'TSTD1', 'PDSS2', 'TRAPPC8', 'SLC38A9', 'PPP1R16B', 'RGS18', 'DOCK8', 'IFIT3', 'CREB3L2', 'SFMBT2', 'CD55', 'CCSER2', 'AGPAT5', 'MRPL28', 'ABLIM1', 'TNFRSF1B', 'LINC02446', 'RAP1GDS1', 'FBXL20', 'AQP3', 'TTC7A', 'WRN', 'ZBP1', 'PARP15', 'APC', 'MAST2', 'TATDN3', 'ITPK1', 'TAFA2', 'TCF4', 'GPANK1', 'PAM', 'UBE2E3', 'SESTD1', 'TUBD1', 'FMNL2', 'MFHAS1', 'TSEN2', 'HIST1H2AC', 'CSF2RA', 'MTAP', 'LACTB', 'MVB12B', 'ARHGAP24', 'INSR', 'ISG15', 'IDH3A', 'AHCYL2', 'NSD2', 'ZFYVE28', 'NMT2', 'SLC44A2', 'LARP1B', 'NEK6', 'INVS', 'KIAA0556', 'FBXO22', 'FER', 'CRYZL1', 'DCPS', 'GBP5', 'ERI1', 'MARCKSL1', 'ME2', 'ZNF718', 'LDAH', 'CCDC50', 'TFRC', 'MAP2K6', 'DHRS3', 'RUNX2', 'CASC4', 'KIAA2013', 'RFFL', 'MAP4K3', 'MNDA', 'FGD2', 'QSER1', 'B2M', 'RASA3', 'ST8SIA4', 'SIPA1L3', 'ELP2', 'ATG16L1', 'TOX', 'ZNF831', 'SLC39A9', 'CYFIP2', 'KRI1', 'ATP2B4', 'SARNP', 'NDUFAF6', 'RBL1', 'EPSTI1', 'MPP6', 'LINC00243', 'KATNBL1', 'TOM1', 'SNX10', 'LINC00937', 'SPAG9', 'RPS26', 'TNS3', 'SERPINB9', 'CD7', 'WHAMM', 'ICOS', 'HSPA4', 'AIG1', 'XAF1', 'RMND1', 'DNAJB1', 'ACVR1', 'TEC', 'MAP3K14', 'MEF2C', 'PITPNA', 'EHMT2', 'ADARB1', 'KCNE1', 'LGALS2', 'COMMD1', 'ALOX5AP', 'SNX4', 'GLMN', 'SAMD12', 'USP40', 'DROSHA', 'FAAH2', 'PLD4', 'HIPK2', 'CLIC3', 'PILRA', 'HS3ST3B1', 'ABI2', 'GLT1D1', 'PITPNM2', 'EIF2AK3', 'HABP4', 'UST', 'GPATCH2L', 'MFAP1', 'DMXL2', 'ALKBH7', 'ZNF37A', 'STX6', 'UBE3D', 'KCNQ5', 'CPB2-AS1', 'BCKDHB', 'ADGRE5', 'MRTFB', 'AK9', 'ARMH1', 'UBL3', 'CPEB4', 'IDH1', 'RHBDD1', 'FKTN', 'UBA7', 'ZDHHC6', 'LMNA', 'LRP1', 'PRH1', 'MARCKS', 'SYTL2', 'HIPK3', 'ITGAM', 'ELL2', 'RILPL2', 'SPTLC2', 'IPMK', 'MLLT3', 'HIVEP3', 'MTERF4', 'C1orf56', 'SRGN', 'CPNE8', 'SLC25A25', 'SPIDR', 'CSTA', 'VOPP1', 'RPS27A', 'PLAC8', 'NUP50-DT', 'MFSD11', 'ARL13B', 'SFXN5', 'COG2', 'FBXO33', 'SOS2', 'VCAN', 'LRBA', 'RPL35A', 'MAST4', 'RNF115', 'TOGARAM1', 'C9orf85', 'ITGB7', 'MEF2C-AS1', 'TTC39C', 'CCDC141', 'RNF157', 'NFIA', 'RCN2', 'CCDC124', 'GALNT3', 'SLC25A13', 'XKR6', 'PALM2-AKAP2', 'OSBPL5', 'SRBD1', 'TMEM120B', 'VPS35L', 'TRIM25', 'PPARA', 'PRKACB', 'HDAC9', 'SRD5A1', 'C6orf89', 'TBC1D20', 'VKORC1L1', 'TMEM161B-AS1', 'DAPK1', 'FXN', 'IKZF3', 'ATG2A', 'FAM117B', 'MYO1E', 'SAT1', 'RFX2', 'TECPR1', 'SMARCAL1', 'EVI5', 'TMEM260', 'DUSP10', 'HIP1', 'NUMB', 'SLC25A12', 'MRPL1', 'SEPTIN11', 'TMEM164', 'EPB41L3', 'RPS6KA4', 'LINC00649', 'PIGL', 'AMN1', 'SAMD4A', 'CEACAM21', 'GABPB1', 'BLOC1S4', 'SDCCAG8', 'RORA', 'STMN3', 'APOBEC3A', 'TOR1B', 'CEP295', 'PRKN', 'PSTPIP2', 'ZNF43', 'MAN1A1', 'DDX58', 'USP37', 'DHRS7B', 'NCF1', 'BHLHE40', 'HEATR5A', 'IL4R', 'CBFA2T2', 'CYHR1', 'C9orf72', 'UFC1', 'CRISPLD2', 'VMP1', 'ZNF830', 'PTPN7', 'SASH1', 'ERCC3', 'CRTC3', 'TNFAIP3', 'YPEL2', 'TESPA1', 'AK5', 'YARS', 'CD68', 'NPL', 'ATP8A1', 'PPM1L', 'CSTB', 'MCM5', 'ENSA', 'EIF4G3', 'MAGED2', 'EIF4ENIF1', 'RPS25', 'TRIP4', 'CD86', 'EHBP1', 'MXI1', 'CCDC18', 'ID2', 'CEP78', 'SCIMP', 'GNAQ', 'ATP2B1', 'IGF2BP2', 'FBXL5', 'ANKRD36B', 'MPEG1', 'DNAJA1', 'IRF7', 'TAPT1-AS1', 'CDK14', 'GYPC', 'APOBEC3G', 'PTPRK', 'BRWD3', 'ZNF251', 'TSC22D2', 'LYZ', 'FCMR', 'SKAP1', 'GZMM', 'CD3E', 'GPCPD1', 'SCLY', 'CPEB3', 'VCPKMT', 'DNAJC17', 'PBX3', 'GBP4', 'C11orf80', 'NUBPL', 'CD247', 'GNS', 'NSUN6', 'ATAD2', 'PARP14', 'EXOSC8', 'CEBPD', 'PHTF1', 'MIR646HG', 'WWP2', 'ISCA1', 'MAF', 'PRR14L', 'TENT5C', 'RIN3', 'ZNF277', 'RHOQ', 'CREB5', 'GOLGA8A', 'SH3RF3', 'CCDC88A', 'DOCK7', 'NAA60', 'ADAM8', 'SLAMF6', 'ST3GAL6', 'PPM1K', 'ZMYND11', 'WARS', 'UBXN2A', 'PPP3CA', 'COTL1', 'RASGRP1', 'APP', 'PPCDC', 'B4GALT5', 'SEPTIN1', 'RPL32', 'FNDC3B', 'NUCB2', 'FCER2', 'CTNND1', 'RPS15A', 'EFL1', 'FCER1G', 'CD3D', 'DPH6', 'CDC14A', 'RIN2', 'MOB1B', 'NEGR1', 'ITK', 'IFRD1', 'CD27', 'AGO4', 'EXOSC3', 'ACTN1', 'S100A11', 'SPATS2L', 'KCNQ1OT1', 'BIRC3', 'TBC1D9', 'ATP6V0A1', 'ALOX5', 'NAE1', 'CASS4', 'GLCCI1', 'FBXO34', 'LYST', 'RHOH', 'AP4E1', 'UGGT2', 'CSNK1G3', 'NUAK2', 'FCHSD2', 'PDE8A', 'ACTR1B', 'AOPEP', 'MCCC1', 'GRAP2', 'CCDC167', 'NME7', 'TMED8', 'S100A9', 'PAK1', 'TRPS1', 'DRG1', 'SYNE1', 'RTN3', 'HSH2D', 'RNF6', 'SLC15A4', 'RPL10', 'AP3S2', 'NOL4L', 'TACC1', 'HRH2', 'LPAR1', 'CISH', 'AHNAK', 'ZMAT3', 'PDZD8', 'PLBD1', 'CD244', 'ACSL4', 'PDE5A', 'ZBTB38', 'ATF1', 'SCARB2', 'FCGR1A', 'CD96', 'NAA35', 'NR4A1', 'ABL1', 'GBE1', 'ZNF276', 'EEF1A1', 'CTSW', 'RSAD2', 'KLF12', 'IPO11', 'CCAR1', 'NDFIP1', 'C11orf49', 'IL21R', 'SLAMF7', 'GATA3', 'PYGL', 'ATG7', 'FUT8', 'NOD2', 'HLA-DRA', 'PLEKHA3', 'UBAP1', 'PKD2', 'PID1', 'BCL11B', 'SIMC1', 'RNF24', 'CFP', 'COG6', 'ABTB1', 'AP3M2', 'ARHGEF40', 'TCF7', 'SLC31A2', 'AGPAT4', 'PADI4', 'LCK', 'MAP2K1', 'HIVEP2', 'PPM1D', 'HEG1', 'ESR1', 'MIR4435-2HG', 'RALGAPA2', 'RPS2', 'TMCO4', 'POC1B', 'INPP4B', 'PRKCA', 'PLPP1', 'OSBPL10', 'XYLT1', 'SLF1', 'ST6GALNAC3', 'GRN', 'LRMDA', 'FAR2', 'RASGEF1B', 'ADAM28', 'WDR49', 'ITPKB', 'SGMS2', 'MRPS18B', 'IFIT2', 'ZBTB16', 'CTBP2', 'TFCP2', 'LIMD1', 'MCM9', 'SHISAL2A', 'WDFY3', 'PFKFB4', 'FLVCR2', 'GXYLT1', 'ZRANB3', 'BCR', 'DOCK4', 'RRAS2', 'RPS6KC1', 'NELL2', 'CD79B', 'CPED1', 'ARHGEF10L', 'TNRC6C', 'MTSS1', 'WDFY2', 'SMIM4', 'RASGRF2', 'SLC43A2', 'ALCAM', 'PIBF1', 'NLRP12', 'TRPC4AP', 'TTYH2', 'OFD1', 'FBXO28', 'TIAM1', 'SLC4A7', 'DHX34', 'LINC02245', 'PDE7A', 'IMP3', 'MEAF6', 'DIP2A', 'FOCAD', 'FLNB', 'NDUFV2-AS1', 'TYW1B', 'CCAR2', 'CFD', 'APBA2', 'TRERF1', 'PYHIN1', 'CDA', 'CNKSR2', 'CDK8', 'TC2N', 'ZNF654', 'LIMS1', 'TRIM52', 'EMILIN2', 'CALCOCO1', 'PRKAG2', 'SSR4', 'TMEM223', 'DPYD', 'NEK1', 'RPS18', 'ZSWIM8', 'TMEM192', 'INTS1', 'CLASRP', 'ALPK1', 'COX10-AS1', 'PKP4', 'ANKRD36C', 'C19orf53', 'LILRB2', 'MYC', 'FAM20B', 'ITM2A', 'SLC25A33', 'TTF1', 'ARMC1', 'CRTC1', 'HNMT', 'SH2D3C', 'SCLT1', 'CENPP', 'PDK1', 'SFMBT1', 'NFIL3', 'ZNF611', 'CASTOR3', 'EEPD1', 'KAT6B', 'ING5', 'MTMR3', 'DDX60L', 'LRRC28', 'PELI1', 'ORC5', 'MPP5', 'C1orf52', 'RUBCNL', 'MICAL3', 'RAB27A', 'CD36', 'MAPKAPK5-AS1', 'CD38', 'PATL2', 'UBE2E1', 'SLC9A9', 'PRMT9', 'SLC16A1-AS1', 'L3MBTL4', 'OAS3', 'PWWP3A', 'CALU', 'TLR2', 'LINC01550', 'ITGB1', 'RNF19B', 'DENND3', 'SUSD1', 'BMP2K', 'DZIP3', 'TTBK2', 'MAPK8', 'KIAA0355', 'MECP2', 'APOLD1', 'HACD3', 'TMEM144', 'MTX2', 'METTL15', 'IL1RAP', 'NBEAL1', 'LRRK2', 'LRRK1', 'AHR', 'FANCC', 'NAMPT', 'NBEAL2', 'CMSS1', 'GRIK1', 'ZNF827', 'S100Z', 'RPL23A', 'WDPCP', 'CDR2', 'MATK', 'SYNE2', 'TRAF1', 'KLRG1', 'STAM', 'DDX60', 'TDRD3', 'PIK3IP1', 'SLC2A13', 'IFITM1', 'SGK1', 'MICAL2', 'OSCAR', 'DPH5', 'SLFN12L', 'TRAK1', 'NOL11', 'FBN2', 'P2RX1', 'KMO', 'SLFN11', 'CCDC200', 'CXXC5', 'CRADD', 'POLD3', 'EGLN3', 'TPGS2', 'RTTN', 'JARID2', 'PCNX2', 'NFKBIZ', 'CAMK4', 'LDLR', 'OSBPL2', 'KIAA0825', 'SIL1', 'SNED1', 'AGAP3', 'ABAT', 'INPP5F', 'TMEM170B', 'MT2A', 'TRAPPC13', 'POP5', 'HMGXB4', 'HDAC8', 'CLIP4', 'ABCD2', 'PRKD2', 'NIPSNAP2', 'TPD52', 'PFN1', 'VRK1', 'ZNF75D', 'THEMIS', 'BCL2A1', 'GABPB1-AS1', 'HERC5', 'CD2', 'LY9', 'DNAJC24', 'AFG1L', 'C20orf194', 'SIK2', 'ATXN3', 'ITGB3BP', 'MIB1', 'RBL2', 'CIITA', 'PCSK5', 'LYN', 'CCSER1', 'RASSF1', 'CLMN', 'S100A4', 'HLA-DQB1', 'EFHD2', 'SRGAP2B', 'CMIP', 'CYFIP1', 'KLRK1', 'STX8', 'UVRAG', 'SLC35D1', 'SESN3', 'RGS2', 'KLHDC10', 'DTWD1', 'ASAP1', 'VEGFB', 'PNPT1', 'ATXN7L1', 'KIF13A', 'IFI44L', 'DYNLL1', 'TPMT', 'RAD17', 'ATP8B2', 'LGALS1', 'GPRIN3', 'FBXW8', 'LEF1', 'FAM117A', 'PTCD3', 'CTSZ', 'MEF2A', 'MAPKAPK2', 'PTAFR', 'LRRN3', 'RAB35', 'RGCC', 'GLUL', 'PACS1', 'MED31', 'IMMP1L', 'CPLANE1', 'TASOR', 'PHLDB3', 'SIRPB2', 'RPL19', 'ACTB', 'VIM-AS1', 'LRRC8B', 'HVCN1', 'INTS12', 'KLHL5', 'FASTKD1', 'SEC24A', 'SMS', 'FGGY', 'MAP3K20', 'PDE7B', 'ANKS1A', 'TSPAN5', 'NIPAL2', 'NR4A2', 'C12orf75', 'MYO9B', 'FCAR', 'ECHDC2', 'RELL1', 'STXBP5', 'DGKE', 'TMEM156', 'IFT80', 'SGK3', 'BCL6', 'UBASH3A', 'ARRDC2', 'BCAS3', 'DLEU2', 'DOP1B', 'ERN1', 'RPL31', 'NRIP1', 'TSC22D1', 'IRAK3', 'TYROBP', 'ABCA5', 'CST3', 'MICU1', 'ATG10', 'RNF130', 'ZCCHC10', 'HIST1H4C', 'TENT5A', 'CD81', 'HLA-DQA1', 'MOB3B', 'CHURC1', 'CD69', 'SLC4A8', 'UBE2E2', 'CDIPT', 'TRG-AS1', 'SLC17A5', 'CMTM8', 'KMT2B', 'PPP1R15B', 'GAB2', 'HPS5', 'HOPX', 'IL15RA', 'RPS6', 'ENOX2', 'LRRC25', 'TMCC1', 'BICDL1', 'SPECC1', 'ALMS1', 'GASK1B', 'PASK', 'ABCC4', 'MX2', 'SNTB1', 'RPS3', 'PLAGL1', 'CLIC4', 'POLA1', 'IFITM2', 'ZNF33B', 'SLAMF1', 'CARNMT1', 'STX7', 'EZH2', 'RAB11FIP1', 'TTLL5', 'ANKRD22', 'CAMK2D', 'ENTPD1-AS1', 'MS4A4E', 'ARHGAP21', 'HMGB2', 'SLX4IP', 'TAF4B', 'TYW5', 'SIRPG', 'RPAP2', 'RAPGEF2', 'CTDP1', 'SNRNP27', 'HIBCH', 'NCEH1', 'RIPK2', 'SEL1L3', 'B4GALT1', 'ARHGAP26', 'RABGEF1', 'ARHGEF11', 'LRSAM1', 'PRAG1', 'ARHGEF18', 'LRRFIP2', 'TMUB1', 'ABR', 'SPAST', 'PRKCQ', 'OXR1', 'HMGCS1', 'LUZP1', 'RRP12', 'PTPRJ', 'CD3G', 'VAV2', 'TUBA1A', 'TPT1', 'TPK1', 'DHPS', 'C19orf12', 'TNIK', 'TECPR2', 'MEGF9', 'CEP162', 'CDH23', 'NFAT5', 'IFT74', 'GPR171', 'KANSL1L', 'LINC00662', 'ZNF433-AS1', 'LTBP3', 'ZDHHC17', 'RHOC', 'SELL', 'NCAPD2', 'IRS2', 'CPVL', 'EPB41', 'STK3', 'ORC3', 'LMTK2', 'PIGN', 'TBC1D32', 'TMX4', 'RSU1', 'BTBD11', 'SMIM8', 'SAMSN1', 'ZNF331', 'SIK3', 'CUTC', 'PRF1', 'PLEKHA2', 'SMARCD1', 'TSHZ2', 'SUFU', 'PHLPP1', 'ICAM1', 'MYOF', 'PHF21A', 'MRPS9', 'RORA-AS1', 'RAB20', 'LINC00877', 'HLA-DRB5', 'REEP3', 'DAAM1', 'DCAF10', 'NLN', 'PACS2', 'GCNT2', 'LILRB3', 'NIPAL3', 'SLC10A7', 'NCF2', 'KDM5B', 'NKG7', 'ABHD12', 'ARL4C', 'ENOSF1', 'POLR3C', 'PLAUR', 'RNF144A', 'MCTP1', 'GALNT10', 'TRAT1', 'TEX10', 'EMD', 'APLP2', 'PUDP', 'RNASE6', 'ATP8B1', 'CD6', 'PRKCH', 'TRMT11', 'TET3', 'NCBP2AS2', 'XPNPEP3', 'LRRC37A3', 'SFI1', 'NLRP3', 'RTN1', 'LDHB', 'MYO1F', 'AKT3', 'ACVR2A', 'KSR1', 'CLUAP1', 'MCU', 'LYRM7', 'LY86', 'B3GNTL1', 'METTL8', 'PTPRM', 'SCMH1', 'UMAD1', 'CD74', 'IL7R', 'ABHD3', 'ODC1', 'CPPED1', 'LPCAT1']

GRIPNet_Res = Run_GRIPNet(CD8_scRNA_DN_magic_ad, SID_train, SID_outsample, pbmc_3k_scATAC_Gene_Enhancer_CD8_psum, G_PPI_BioGrid, TF_Motif_Metric[2], TargetGenes = TGList, Lambda = 0.001)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-30
1 / 1846 : HIVEP1 10 ['CREB5', 'ESR1', 'MYC', 'HIF1A', 'SMAD3', 'SPI1', 'MECP2', 'ATF3', 'RXRA', 'ESR2'] 0.0055698702879047905
2 / 1846 : CYBB 5 ['CREB5', 'ESR1', 'MECP2', 'RXRA', 'ESR2'] 0.0013828146698237258
3 / 1846 : ANKRD26 5 ['CREB5', 'BCL6', 'ESR1', 'RELB', 'MECP2'] 0.003953640813308234
4 / 1846 : IL32 6 ['ESR1', 'FOXO1', 'MYC', 'FOXO3', 'CEBPD', 'SMAD3'] 0.019185103260435696
5 / 1846 : ZC4H2 9 ['ZEB1', 'TAF1', 'KLF4', 'KLF12', 'RUNX2', 'HLTF', 'PPARD', 'E2F3', 'RUNX3'] 0.011283387681722094
6 / 1846 : PLXNC1 8 ['JUN', 'MYC', 'ESR1', 'SMAD3', 'MECP2', 'RXRA', 'SPI1', 'ESR2'] 0.008425416254738563
7 / 1846 : AAMDC 9 ['SMAD3', 'RXRA', 'MECP2', 'ESR2', 'NR4A1', 'FOXO3', 'BCL6', 'FOXO1', 'NR2C2'] 0.009745814791122576
8 / 1846 : HLA-DRB1 10 ['MYC', 'ESR1', 'SMAD3', 'JUN', 'CEBPB', 'MECP2', 'RXRA', 'ESR2', 'FOS', 'NR4A1'] 0.006734448469264142
9 / 1846 : SFXN1 10 ['ESR1', 'HIF1A', 'SMAD3', 'JUN', '

/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


59 / 1846 : RPS5 10 ['MYC', 'ESR1', 'HIF1A', 'JUN', 'SMAD3', 'MECP2', 'ESR2', 'FOS', 'RXRA', 'FOXO3'] 0.010182740429832299
60 / 1846 : ZSWIM6 9 ['ESR1', 'JUN', 'CEBPB', 'CREB5', 'ESR2', 'RXRA', 'FOS', 'NR4A1', 'BCL6'] 0.006078404078407778
61 / 1846 : BATF 3 ['BATF', 'ESR1', 'MYC'] 8.410747427683533e-07
62 / 1846 : JMJD1C 10 ['ESR1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'FOS', 'RXRA', 'ESR2', 'NR4A1', 'KLF4'] 0.009240982702425182
63 / 1846 : CC2D2A 10 ['ESR1', 'MYC', 'HIF1A', 'SMAD3', 'ESR2', 'RXRA', 'NR4A1', 'FOXO1', 'BACH1', 'TAF1'] 0.004558279246375874
64 / 1846 : ZNF487 10 ['MYC', 'ESR1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'ATF3', 'CEBPB', 'RXRA', 'NR2C2'] 0.008874441596208475
65 / 1846 : PSIP1 8 ['MYC', 'ESR1', 'JUN', 'HIF1A', 'SMAD3', 'MECP2', 'BCL6', 'ESR2'] 0.014918852636509506
66 / 1846 : SMAD3 4 ['SMAD3', 'MYC', 'ESR1', 'HIF1A'] 9.157158833925351e-07
67 / 1846 : PLD1 5 ['CEBPB', 'ESR1', 'NR4A1', 'MYC', 'IRF7'] 0.007421727676766335
68 / 1846 : ZDHHC20 10 ['ESR1', 'MYC', 'SMAD3', 'H

/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


103 / 1846 : UGCG 9 ['ESR1', 'MECP2', 'RXRA', 'ESR2', 'NR4A1', 'FOXO3', 'NR2C2', 'BACH1', 'TAF1'] 0.007078418893573023
104 / 1846 : STK25 10 ['MYC', 'ESR1', 'SMAD3', 'HIF1A', 'JUN', 'MECP2', 'CEBPB', 'ESR2', 'NR2C2', 'RXRA'] 0.011103157768407066
106 / 1846 : MINDY2 10 ['ESR1', 'MYC', 'KLF4', 'HIF1A', 'SMAD3', 'MECP2', 'CEBPB', 'RXRA', 'ESR2', 'NR2C2'] 0.010627742622447815
107 / 1846 : FILIP1L 10 ['MYC', 'SMAD3', 'MECP2', 'CEBPB', 'RXRA', 'ESR2', 'NR4A1', 'BCL6', 'CREB5', 'FOXO1'] 0.006184488542410053
108 / 1846 : SYNJ2 10 ['MYC', 'ESR1', 'SMAD3', 'HIF1A', 'JUN', 'MECP2', 'RXRA', 'ESR2', 'FOS', 'NR4A1'] 0.008477934329019138
109 / 1846 : ACACA 3 ['IRF8', 'KLF4', 'FOXO1'] 0.010321250227753626
110 / 1846 : LANCL1 8 ['ESR1', 'CEBPB', 'RXRA', 'ESR2', 'BCL6', 'KLF4', 'MEF2C', 'NR2C2'] 0.014056774044649884
111 / 1846 : P2RX7 7 ['HIF1A', 'ESR1', 'NR4A1', 'MYC', 'JUN', 'MECP2', 'SMAD3'] 0.005797605745398395
112 / 1846 : CST7 10 ['CREB5', 'KLF4', 'BACH1', 'JUN', 'MECP2', 'MYC', 'ESR1', 'ESR2', 'R

...

1737 / 1846 : CD3G 10 ['BCL6', 'ESR1', 'HIF1A', 'SPI1', 'MYC', 'SMAD3', 'JUN', 'NR4A1', 'MECP2', 'RXRA'] 0.023971639927216713
1738 / 1846 : VAV2 10 ['ESR1', 'MYC', 'HIF1A', 'SMAD3', 'MECP2', 'ESR2', 'RXRA', 'NR4A1', 'BCL6', 'FOXO3'] 0.0023341820758404504
1739 / 1846 : TUBA1A 9 ['ESR1', 'MYC', 'SMAD3', 'HIF1A', 'JUN', 'CEBPB', 'MECP2', 'KLF4', 'NR4A1'] 0.006799304696227095
1740 / 1846 : TPT1 8 ['MYC', 'ESR1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'IRF7', 'MEF2C'] 0.011691531326503558
1741 / 1846 : TPK1 3 ['ETV6', 'RELB', 'ZNF76'] 0.0060573345662791045
1742 / 1846 : DHPS 10 ['MYC', 'ESR1', 'SMAD3', 'HIF1A', 'JUN', 'CEBPB', 'MECP2', 'ESR2', 'RXRA', 'FOS'] 0.012020447646129744
1743 / 1846 : C19orf12 8 ['ESR1', 'MYC', 'HIF1A', 'JUN', 'SMAD3', 'RXRA', 'MECP2', 'MXI1'] 0.00836894230230603
1744 / 1846 : TNIK 7 ['SPI1', 'CEBPB', 'ESR1', 'NR4A1', 'MECP2', 'SMAD3', 'TAF1'] 0.011427518319139534
1745 / 1846 : TECPR2 5 ['CREB5', 'KLF4', 'HIF1A', 'JUN', 'BACH1'] 0.007424153101494475
1746 / 1846 : MEGF9 1

/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


1796 / 1846 : LILRB3 3 ['NR4A1', 'KLF4', 'CEBPB'] 0.0021075717628766225
1797 / 1846 : NIPAL3 10 ['ESR1', 'NR4A1', 'HIF1A', 'SMAD3', 'JUN', 'CEBPB', 'MECP2', 'RXRA', 'ESR2', 'KLF4'] 0.012603421992213688
1798 / 1846 : SLC10A7 10 ['ESR1', 'SMAD3', 'MECP2', 'ESR2', 'RXRA', 'NR4A1', 'BCL6', 'TAF1', 'SPI1', 'KLF12'] 0.01065742881650023
1799 / 1846 : NCF2 9 ['ESR1', 'MYC', 'MECP2', 'CEBPB', 'ESR2', 'RXRA', 'FOS', 'CREB5', 'NR4A1'] 0.0035794995288458325
1800 / 1846 : KDM5B 6 ['NR4A1', 'PPARD', 'ATF3', 'PAX5', 'JUN', 'KLF4'] 0.008334892838058506
1801 / 1846 : NKG7 7 ['SPI1', 'RXRA', 'PAX5', 'FOS', 'JDP2', 'FOXO1', 'ATF3'] 0.028273955229647558
1802 / 1846 : ABHD12 5 ['CREB5', 'SMAD3', 'RXRA', 'ETV6', 'MECP2'] 0.008279900512688567
1803 / 1846 : ARL4C 6 ['JDP2', 'KLF4', 'CREB5', 'SPI1', 'CEBPD', 'ESR2'] 0.01841013412584812
1804 / 1846 : ENOSF1 9 ['ESR1', 'HIF1A', 'SMAD3', 'JUN', 'MECP2', 'CEBPB', 'SPI1', 'NR4A1', 'KLF4'] 0.007175770952262105
1805 / 1846 : POLR3C 6 ['ESR1', 'CREB5', 'RXRA', 'FOXO3'

/Users/yijwang-admin/Documents/Research/GRN_PPI/L0GL_PPI_GRN/minFunc/polyinterp.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  t = points[notMinPos, 0] - (points[notMinPos, 0] - points[minPos, 0]) * ((points[notMinPos, 2] + d2 - d1) / (points[notMinPos, 2] - points[minPos, 2] + 2 * d2))


1809 / 1846 : GALNT10 10 ['MYC', 'ESR1', 'SMAD3', 'JUN', 'NR4A1', 'MECP2', 'CEBPB', 'RXRA', 'ESR2', 'FOS'] 0.011071115604346711
1810 / 1846 : TRAT1 10 ['ESR1', 'KLF4', 'SMAD3', 'JUN', 'MECP2', 'RXRA', 'SPI1', 'ESR2', 'FOS', 'BCL6'] 0.015705807815398355
1811 / 1846 : TEX10 6 ['PAX5', 'CREB5', 'ATF3', 'ESR1', 'MAFB', 'KLF12'] 0.009329204430432114
1812 / 1846 : EMD 8 ['CREB5', 'KLF4', 'ESR1', 'MEF2C', 'MYC', 'CEBPB', 'ESR2', 'MAFB'] 0.012207625899432143
1813 / 1846 : APLP2 10 ['MYC', 'ESR1', 'HIF1A', 'SMAD3', 'MECP2', 'CEBPB', 'RXRA', 'ESR2', 'KLF4', 'NR2C2'] 0.005349381364462513
1814 / 1846 : PUDP 6 ['MYC', 'ESR1', 'JUN', 'MECP2', 'SPI1', 'ESR2'] 0.0053906923535360215
1815 / 1846 : RNASE6 3 ['SPI1', 'PAX5', 'RXRA'] 0.0018903191089032385
1816 / 1846 : ATP8B1 9 ['HIF1A', 'ESR1', 'SMAD3', 'JUN', 'MECP2', 'SPI1', 'KLF4', 'ESR2', 'RXRA'] 0.006491583420826708
1817 / 1846 : CD6 6 ['NR4A1', 'MECP2', 'BCL6', 'MYC', 'HIF1A', 'SPI1'] 0.01781972313310541
1818 / 1846 : PRKCH 6 ['NR4A1', 'KLF12', 'BCL

In [22]:
GRIPNet_Res

,TG,TFs,Out_of_Sample MSE
0,HIVEP1,"[CREB5, ESR1, MYC, HIF1A, SMAD3, SPI1, MECP2, ...",0.005570
1,CYBB,"[CREB5, ESR1, MECP2, RXRA, ESR2]",0.001383
2,ANKRD26,"[CREB5, BCL6, ESR1, RELB, MECP2]",0.003954
3,IL32,"[ESR1, FOXO1, MYC, FOXO3, CEBPD, SMAD3]",0.019185
4,ZC4H2,"[ZEB1, TAF1, KLF4, KLF12, RUNX2, HLTF, PPARD, ...",0.011283
...,...,...,...
1760,IL7R,"[CREB5, ESR1, KLF4, MYC, BCL6, ATF3, SMAD3, CE...",0.023530
1761,ABHD3,"[ESR1, HIF1A, SMAD3, MECP2, CEBPB, RXRA, ESR2,...",0.005617
1762,ODC1,"[MYC, ESR1, HIF1A, SMAD3, JUN, MECP2, CEBPB, E...",0.008130
1763,CPPED1,"[MECP2, ESR2, FOXO1, TAF1, SPI1, KLF12, TFCP2,...",0.003918


In [23]:
GRIPNet_Res.to_pickle('PBMC3K_CD8_GRIP_Gene1765_Result_Motifp1e-4_lambda1e-3_pho1e-4.pkl')

In [ ]:
TGList = ['GNPTAB', 'MINK1', 'CAAP1', 'FGGY', 'CISH', 'PRIM2', 'DLG2', 'TTLL3', 'KLHL15', 'DNAJA1', 'SLC38A1', 'PPARD', 'LATS1', 'BLK', 'DDHD2', 'VPS45', 'N4BP2', 'KATNBL1', 'NEDD9', 'DUSP16', 'JARID2', 'DPP4', 'TGIF1', 'THEM4', 'CMIP', 'CREB5', 'MED19', 'PIK3CA', 'HEG1', 'IFT74', 'HSPA1A', 'SNX10', 'APLP2', 'TDP2', 'CD68', 'GSR', 'SELL', 'ZNF487', 'APC', 'SCNM1', 'IFITM1', 'MAP4K5', 'GCH1', 'PRDM1', 'POU2F1', 'ARAP2', 'IFI6', 'RCAN3', 'ZBTB16', 'BLOC1S4', 'KPNA5', 'PSMA1', 'GNB5', 'FLNB', 'CTDSPL2', 'VPS16', 'ITGA6', 'ASH2L', 'PIM1', 'NFRKB', 'HEATR3', 'CDIPT', 'KDM1B', 'NUP160', 'R3HDM1', 'MRPL28', 'SPECC1', 'CD5', 'RSPRY1', 'PHACTR4', 'LPCAT1', 'CCDC66', 'HOOK2', 'NFAM1', 'ANGPT2', 'NUDCD3', 'SIK3', 'PELI1', 'CTC1', 'TFEC', 'SSRP1', 'RPS6', 'SLAMF1', 'NFX1', 'NNT', 'FNDC3B', 'PTCD3', 'SPATA2L', 'CCND3', 'RIC8B', 'RPS15A', 'MTAP', 'MFHAS1', 'TMEM14B', 'AHNAK', 'RBM41', 'SNAPC5', 'INTS3', 'LRIG1', 'RMDN3', 'COQ7', 'UBA3', 'JAK2', 'CHD4', 'IL2RB', 'DGKA', 'SERTAD2', 'TSTD1', 'PTPRJ', 'ATP2B4', 'CSGALNACT1', 'C12orf57', 'ZRANB3', 'MRPL42', 'POLM', 'SNRK', 'PACS1', 'C11orf65', 'KCNQ1', 'ST6GAL1', 'HPS3', 'RPL31', 'SNAPC1', 'NUP107', 'SSBP3', 'CCDC167', 'MCM9', 'ABR', 'SLCO3A1', 'CEP68', 'ITGAE', 'PATL1', 'CHMP7', 'LRRC8C', 'AQP3', 'PDK1', 'LRP1', 'PRR5L', 'COA1', 'ING4', 'TTC39B', 'DCPS', 'ZDHHC5', 'SLC44A2', 'METTL21A', 'TET3', 'UBASH3B', 'CCAR2', 'IGF1R', 'ATXN1', 'CSPP1', 'FCER2', 'TESK2', 'LEPROTL1', 'IFT88', 'RIN3', 'DOCK8', 'ICAM1', 'TNFRSF1B', 'PIK3IP1', 'PRKCQ-AS1', 'SENP1', 'SCML4', 'CEACAM21', 'TNFAIP3', 'MCM3', 'LRRC23', 'TIPARP', 'HIVEP1', 'PSIP1', 'IMP3', 'IL21R', 'ITGAX', 'COTL1', 'SIK2', 'ZNF263', 'RBM10', 'SEH1L', 'TNIK', 'RPS14', 'INPP5K', 'RPS27', 'IL7R', 'CDC42EP3', 'ACTB', 'NUP43', 'PLIN2', 'ATP8A1', 'METTL16', 'TRERF1', 'SREK1IP1', 'TRABD2A', 'RHOH', 'ZMAT3', 'TBC1D14', 'HLA-DMB', 'BTBD2', 'APH1B', 'PIGL', 'AHR', 'COX10', 'FCGR2A', 'LCMT1', 'TTC4', 'TNFRSF10A', 'RELL1', 'LPCAT3', 'UXS1', 'COMMD9', 'ZBTB38', 'SYNE2', 'RPL3', 'CAMK1D', 'DHRS3', 'ODC1', 'NFATC2IP', 'NVL', 'IL4R', 'SLC30A7', 'LITAF', 'RPS26', 'TMEM238', 'AP4E1', 'PSMD6-AS2', 'VOPP1', 'ARHGEF11', 'SOS2', 'CD55', 'TDRD3', 'RAI1', 'UBE2E3', 'NUBPL', 'CD3E', 'KIF16B', 'CD247', 'AOAH', 'MT1X', 'DHRS4-AS1', 'ZNF789', 'SLC25A42', 'GATAD2B', 'E2F3', 'MAPKAPK2', 'ITGB1', 'MLLT3', 'GSE1', 'GBP5', 'ANKH', 'COG4', 'DHRS7B', 'NUAK2', 'RNF144A', 'HIVEP2', 'EZH2', 'SREBF1', 'CCND2', 'LAP3', 'BIRC3', 'OARD1', 'FGD2', 'CHIC2', 'CHPT1', 'FARS2', 'EMG1', 'RRM1', 'APBB1', 'CD27', 'LCORL', 'TMEM106B', 'ZFYVE28', 'TBL1X', 'RPS3', 'S100A4', 'IL6ST', 'LUZP1', 'ATR', 'UFC1', 'LY9', 'MPHOSPH9', 'SECISBP2L', 'HMGCS1', 'CX3CR1', 'ARHGEF18', 'FOXO1', 'DUSP6', 'SPTBN1', 'SFMBT2', 'LDLRAD4', 'DDX55', 'B4GALT1', 'RAB35', 'ATP13A1', 'UVSSA', 'TRNAU1AP', 'C9orf72', 'NUMB', 'RNF31', 'ITFG2', 'LIMK2', 'SLAMF7', 'SNX2', 'RCL1', 'RAB28', 'ETS1', 'RALGPS2', 'CBFB', 'VPS26B', 'NUP153', 'APOLD1', 'MPV17', 'GPATCH4', 'CLEC7A', 'GINM1', 'SYTL2', 'ENSA', 'CTNND1', 'ZFX', 'FTL', 'NFKBIZ', 'TFRC', 'SLC35E3', 'WBP1', 'GNL1', 'SIAH1', 'APBA2', 'FHIT', 'SESN3', 'WDR59', 'HLA-DRA', 'NXPE3', 'P2RX1', 'MCPH1', 'CD69', 'CD6', 'APIP', 'SLC12A6', 'KRI1', 'CDR2', 'LMNA', 'GPCPD1', 'FOS', 'GLCCI1', 'HK2', 'ARHGAP5', 'VCPKMT', 'TXK', 'ITPKB', 'MCCC1', 'BHLHE40', 'TRDMT1', 'NCAPD2', 'SUPT3H', 'TMEM204', 'PRMT3', 'CREBL2', 'MLH1', 'SERPINB9', 'MYLIP', 'DUS2', 'NPAT', 'AP3M2', 'DHX16', 'RBL2', 'NEAT1', 'SPTY2D1', 'SLMAP', 'RRBP1', 'STK17A', 'PEX1', 'SLAMF6', 'PAM16', 'TRANK1']


GRIPNet_Res = Run_GRIPNet(CD4_scRNA_DN_magic_ad, SID_train, SID_outsample, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, G_PPI_BioGrid, TF_Motif_Metric[2], TargetGenes = TGList)

In [ ]:
GRIPNet_Res, = Run_GRIPNet(CD4_scRNA_DN_magic_ad, SID_train, SID_outsample, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, G_PPI_BioGrid, TF_Motif_Metric[1], TargetGenes = TGList)

In [ ]:
Gene_in_CD4_scRNA.index('TMLHE-AS1')

In [ ]:
Fname = "./CD4_GRIP_Result_Tmp.txt"

GRIPNet_DF = pd.DataFrame(columns=['TG', 'TFs', 'Out_of_Sample MSE'])
fh = open(Fname, 'r')
for line in fh:
    linet = line.strip().split(' ')
    if linet[0] == 'Fail':
        continue
    else:
        TT = line.split(' ')
        Str = ''
        for i in range(2, len(TT)-1):
            #print(TT[i])
            Str = Str + TT[i]
        TFList = eval(Str)
        #print(linet[-1])
        GRIPNet_DF = GRIPNet_DF._append({'TG': linet[0], 'TFs': TFList, 'Out_of_Sample MSE': float(linet[-1])}, ignore_index=True)

        


In [ ]:
GRIPNet_DF

In [ ]:
Fname = "./CD4_GRIP_Result_Motifp1e-4_lambda1e-4_pho1e-4.txt"

GRIPNet_DF = pd.DataFrame(columns=['TG', 'TFs', 'Out_of_Sample MSE'])
fh = open(Fname, 'r')
for line in fh:
    linet = line.strip().split(' ')
    if linet[0] == 'Fail':
        continue
    else:
        TT = line.split(' ')
        Str = ''
        for i in range(2, len(TT)-1):
            #print(TT[i])
            Str = Str + TT[i]
        TFList = eval(Str)
        #print(linet[-1])
        GRIPNet_DF = GRIPNet_DF._append({'TG': linet[0], 'TFs': TFList, 'Out_of_Sample MSE': float(linet[-1])}, ignore_index=True)

        

In [ ]:
GRIPNet_DF

In [ ]:
GRIPNet_DF.to_pickle('PBMC3K_CD4_GRIP_Result_Motifp1e-4_lambda1e-4_pho1e-4.pkl')

In [ ]:
TF_Inferred = list(set(itertools.chain.from_iterable(GRIPNet_DF['TFs'].tolist())))
print(len(TF_Inferred))

In [ ]:
GRIPNet_DF.to_pickle('PBMC3K_CD4_GRIP_Result.pkl')

In [ ]:
Tmp_Input = pbmc_3k_scATAC_Gene_Enhancer_CD4_psum[pbmc_3k_scATAC_Gene_Enhancer_CD4_psum['Genes_within_250k'].apply(lambda x: any(gene in x for gene in ['RPS27']))].reset_index()



In [ ]:
Tmp_Input

## 1.3 Load PCHiC ground truth and trim to CD4

In [ ]:
#load the data
PCHiC_Overlap_Pbmc_3k_AllOP = pd.read_pickle('../../Data/GroundTruth/pbmc_3k_PCHiC/PCHiC_Overlap_Pbmc_3k_ATAC_All_OpenRegion.pkl')


In [ ]:
PCHiC_Overlap_Pbmc_3k_AllOP

In [ ]:
print(list(PCHiC_Overlap_Pbmc_3k_AllOP.columns)[-15:])

In [ ]:
Df_tmp = PCHiC_Overlap_Pbmc_3k_AllOP[['chr_position', 'nCD4', 'baitNameList', 'oeChr', 'oeStart', 'oeEnd', 'Hit']]
Df_tmp = Df_tmp[Df_tmp['nCD4']>=5]
CD4_PCHiC = Df_tmp[Df_tmp['Hit']>=1]

In [ ]:
CD4_PCHiC

In [ ]:
Gene_in_PCHiC = DF_Column_to_List(CD4_PCHiC, 'baitNameList')
print(len(Gene_in_PCHiC))

In [ ]:
OP_Overlap_PCHiC = DF_Column_to_List(CD4_PCHiC, 'chr_position')
print(len(OP_Overlap_PCHiC))

Gene_with_GT = list()
for op in OP_Overlap_PCHiC:
    Gene_with_GT.append(op.split('_')[0])

In [ ]:
Gene_in_CD4_scATAC_scRNA_PCHiC = list(set(Gene_with_GT) & set(Gene_in_CD4_scATAC_scRNA))
print('Gene appears in both scRNA, scATAC, and PCHiC is', len(Gene_in_CD4_scATAC_scRNA_PCHiC))

## 1.4 Load PPI

In [ ]:
G_PPI_BioGrid = pickle.load(open('../../Data/PPI/PPI_BioGrid_Evd2_DSD.pickle', 'rb'))
print("The PPI netowrk has", len(G_PPI_BioGrid.nodes), "nodes and", len(G_PPI_BioGrid.edges), "edges")

# 2. Prepare data for Alg.

In [ ]:
Gene_scRNA = CD4_scRNA_DN_magic_ad.var.index.tolist()
print(len(Gene_scRNA))

# intersection between genes in scRNA and PCHiC ground truth
Gene_Validate = Gene_in_CD4_scATAC_scRNA_PCHiC

print("We focus on", len(Gene_Validate), "genes")

In [ ]:
TargetG = Gene_Validate[1]
print(TargetG)

In [ ]:
Gene_with_GT = list()
Map_Gene_GTOpenR = {}
for i in range(len(Gene_Validate)):
    GL = [Gene_Validate[i]]
    #print(GL)
    Tmp_Input = pbmc_3k_scATAC_Gene_Enhancer_CD4_psum[pbmc_3k_scATAC_Gene_Enhancer_CD4_psum['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()

    Tmp_GT = CD4_PCHiC[CD4_PCHiC['baitNameList'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    
    GTtmp =  Tmp_GT['chr_position'].tolist()
    Input_tmp = Tmp_Input['chr_position'].tolist()

    NumGT = 0
    OPList = list()
    for tt in GTtmp:
        tsp = tt[0].split('_', 1)
        chrp = tsp[1]#+'_'+tsp[2]+'_'+tsp[3]
        #print(tsp, chrp)
        if tsp[0] == GL[0] and chrp in Input_tmp:
            NumGT = NumGT + 1
            OPList.append(chrp)
    
    if NumGT > 0:
        Gene_with_GT.append(GL[0])
        Map_Gene_GTOpenR[GL[0]] = list(set(OPList))
        print(GL[0], "Num OP is", Tmp_Input.shape[0], "Num GT in those OP is", len(set(OPList)))



In [ ]:
print(list(Map_Gene_GTOpenR.keys()))


In [ ]:
# Set up Objective Function
funObj = lambda w: L0GLObj(w, X, y, pho)

funObj_PPI = lambda w: L0GL_PPI_Obj(w, X, y, S, pho, lamb)

# Set up Simplex Projection Function
funProj = lambda w: ProjCSimplex(w, k)

funProj_GL = lambda w: ProjCSimplexGL_Gurobi(w, k, Group, h)

funProj_GL_LB = lambda w: ProjCSimplexGL_Gurobi_LB(w, k, Group, h)

In [ ]:
ColSelect = 'TF_Matching_p-value_1e-05'
lamb = 0.01
pho = 0.01#np.sqrt(NumSample)
h = 1
#Group = [range(NumFeature)]#GroupInd#
options = {'maxIter': 400, 'verbose': 0}

Num_Our_better = 0
for gt in Gene_with_GT:
    GL = [gt]
    TF_Group, TmpInput = Get_TFs_in_Data_noPrint(gt, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, Gene_scRNA)
    
    TFList_ = list(set(list(itertools.chain.from_iterable(TF_Group))))#DF_Column_to_List(Tmp_Input, ColSelect)
    TFList_in_Data = list(set(TFList_) & set(Gene_scRNA))
    
    NumFeature = len(TFList_in_Data)
    NumSample = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X.shape[0]
    Group = [range(NumFeature)]
    #print(NumFeature)

    # Input data
    Xt = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X
    yt = CD4_scRNA_DN_magic_ad[:,GL].X
    SampleIDs = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].obs.index.tolist()

    uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
    #print(uSimplex.shape)

    S =  Short_Path_DSD_Matrix(G_PPI_BioGrid, TFList_in_Data)#Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)

    LSE_Tmp_TFGraph = np.zeros(5)
    LSE_Tmp_No_TFGraph = np.zeros(5)
    k = 4#int(0.1*NumFeature)
    for t in range(5):
        X, y, X_outsample, y_outsample, _, _ = Split_Sample(Xt, yt, SampleIDs, 0.9)
    
        # parameters that can be changed
        lamb = 0.01
        
        uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
        uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
        #print(zout_ppi.flatten())
        SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)
        
        ut = np.zeros((NumFeature,1))
        ut[SelectID,0] = 1.0

        ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
        #print("Obj:", ObjF)
        #print("Short Dis;", ut.T@S@ut)


        XSelect = X[:,SelectID]
        w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(SelectID))) @ XSelect.T @ y
        #print(TF_Select)
        #print(np.abs(w_est.flatten()))

        X_outsample_select = X_outsample[:, SelectID]
        LSE_Tmp_TFGraph[t] = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
    
        # parameters that can be changed
        lamb = 0.0
    
        uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
        uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
        #print(zout_ppi.flatten())
        SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)
        
        ut = np.zeros((NumFeature,1))
        ut[SelectID,0] = 1.0

        ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
        #print("Obj:", ObjF)
        #print("Short Dis;", ut.T@S@ut)


        XSelect = X[:,SelectID]
        w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(SelectID))) @ XSelect.T @ y
        #print(TF_Select)
        #print(np.abs(w_est.flatten()))

        X_outsample_select = X_outsample[:, SelectID]
        LSE_Tmp_No_TFGraph[t] = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
    

    if np.mean(LSE_Tmp_TFGraph) >= np.mean(LSE_Tmp_No_TFGraph):
        print('-', gt, np.mean(LSE_Tmp_TFGraph), np.std(LSE_Tmp_TFGraph),  np.mean(LSE_Tmp_No_TFGraph), np.std(LSE_Tmp_No_TFGraph))
    else:
        print('+', gt, np.mean(LSE_Tmp_TFGraph), np.std(LSE_Tmp_TFGraph),  np.mean(LSE_Tmp_No_TFGraph), np.std(LSE_Tmp_No_TFGraph))
    
    
    if np.mean(LSE_Tmp_TFGraph) < np.mean(LSE_Tmp_No_TFGraph):
        Num_Our_better = Num_Our_better + 1

print(Num_Our_better," out of ", len(Gene_with_GT), "our new model is better!")

In [ ]:
TG = 'TIPARP'
ColSelect = 'TF_Matching_p-value_1e-05'

GL = [TG]

TF_Group, TmpInput = Get_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, Gene_scRNA)
#print(TF_Group)

TFList_ = list(set(list(itertools.chain.from_iterable(TF_Group))))#DF_Column_to_List(Tmp_Input, ColSelect)
TFList_in_Data = list(set(TFList_) & set(Gene_scRNA))
print(TFList_)
print(TFList_in_Data)

GroupInd = Group_Ind(TFList_in_Data, TF_Group)
#print(GroupInd)

{'CEBPD', 'FOS', 'KLF4'}

In [ ]:
NumFeature = len(TFList_in_Data)
NumSample = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X.shape[0]
print(NumFeature)

# Input data
Xt = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].X
yt = CD4_scRNA_DN_magic_ad[:,GL].X
SampleIDs = CD4_scRNA_DN_magic_ad[:,TFList_in_Data].obs.index.tolist()

X, y, X_outsample, y_outsample, SampleID_train, SampleID_outofsample = Split_Sample(Xt, yt, SampleIDs, 0.9)

uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
print(uSimplex.shape)

S =  Short_Path_DSD_Matrix(G_PPI_BioGrid, TFList_in_Data)#Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)
#ShortPath = ShorestPath_GivenTFs(G_PPI_BioGrid, TFList_in_Data, "TestShortPath.txt")
print(S.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
plt.imshow(S, cmap='hot')
plt.xticks(range(len(TFList_in_Data)), TFList_in_Data)
plt.colorbar()
plt.show()


In [ ]:
lamb = 0.001
pho = 0.000001#np.sqrt(NumSample)
h = 1
Group = [range(NumFeature)]#GroupInd#

k = 6
h = 1
      
# parameters that can be changed
options = {'maxIter': 400, 'verbose': 1}

K_Range = range(3,10)
MSE_Our = np.zeros(len(K_Range))
for i in range(len(K_Range)):
    k = K_Range[i]
    
    uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
    uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
    #print(zout_ppi.flatten())
    SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)

    TF_Select = list()
    for idt in SelectID:
        TF_Select.append(TFList_in_Data[idt])
    print(TF_Select)


    MSE_Our[i] = Out_of_Sample_MSE(CD4_scRNA_DN_magic_ad, TG, TF_Select, SampleID_train, SampleID_outofsample)
    print(k, MSE_Our[i])


In [ ]:
TF_Select_scenic = ['CREB5', 'BCL11B', 'KLF4', 'SPI1', 'FOS']
print(TF_Select_scenic)
MSE_scenic = Out_of_Sample_MSE(CD4_scRNA_DN_magic_ad, TG, TF_Select_scenic, SampleID_train, SampleID_outofsample)
print(MSE_scenic)

In [ ]:
import pwlf

my_pwlf = pwlf.PiecewiseLinFit(K_Range, MSE_Our)
breaks = my_pwlf.fit(2)
print(breaks)

x_hat = np.linspace(K_Range[0], K_Range[-1], 100)
y_hat = my_pwlf.predict(x_hat)

plt.figure()
plt.plot(K_Range, MSE_Our, 'o')
plt.plot(x_hat, y_hat, '-')
plt.show()

In [ ]:
from scipy import optimize
def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x <= x0], [lambda x:k1*x + y0+k1*x0, lambda x:k1*x0 + y0-k1*x0])

p , e = optimize.curve_fit(piecewise_linear, K_Range, MSE_Our)
print(p)

In [ ]:
xd = np.linspace(3, 9, 100)
plt.plot(K_Range, MSE_Our, "o")
plt.plot(xd, piecewise_linear(xd, *p))

In [ ]:
TF_Select = list()
for idt in SelectID:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
#print(SelectID_Best)
TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

TF_Select = TF_Select_scenic
print(TF_Select)
TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
GS = GL  + (TF_Select)
yt = CD4_scRNA_DN_magic_ad[:,GS].X

fig, ax = plt.subplots(nrows=len(GS), ncols=1, sharex=True, figsize=(12, 10))
for i in range(len(GS)):
    ax[i].plot(yt[:,i])
    ax[i].set_title(GS[i])


In [ ]:
St =  Short_Path_DSD_Matrix(G_PPI_BioGrid, ['IRF8', 'MEF2C', 'HES1'])#Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)


fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(St, cmap='hot')
plt.xticks(range(len(TF_Select)), TF_Select)
plt.colorbar()
plt.show()

In [ ]:
lamb = 0.0
pho = 0.1#np.sqrt(NumSample)
h = 1
Group = [range(NumFeature)]#GroupInd#

uSimplex = np.ones((NumFeature, 1)) * (1 / NumFeature)
print(uSimplex.shape)

S = Short_Path_Matrix(G_PPI_BioGrid, TFList_in_Data)
ShortPath = ShorestPath_GivenTFs(G_PPI_BioGrid, TFList_in_Data, "TestShortPath.txt")
print(S.shape)

NumItr = 10
k_range = range(3,10)
#k = 6
h_range = range(1,2)
SelectID_Best = ()
LSE_best = 10000
for k in k_range:
    for h in h_range:
        LSE_Tmp = np.zeros(NumItr)
        for i in range(NumItr):
            X, y, X_outsample, y_outsample = Split_Sample(Xt, yt, 0.9)
            
            # parameters that can be changed
            options = {'maxIter': 400, 'verbose': 1}
    
            uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)
    
            uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
            #print(zout_ppi.flatten())
            SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)
        
            ut = np.zeros((NumFeature,1))
            ut[SelectID,0] = 1.0

            ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
            #print("Obj:", ObjF)
            #print("Short Dis;", ut.T@S@ut)


            XSelect = X[:,SelectID]
            w_est = np.linalg.pinv(XSelect.T @ XSelect + pho*np.eye(len(SelectID))) @ XSelect.T @ y
            #print(TF_Select)
            #print(np.abs(w_est.flatten()))

            X_outsample_select = X_outsample[:, SelectID]
            LSE_Tmp[i] = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
        LSE = np.mean(LSE_Tmp)    
        print('k=',k, 'h=', h,"Out-of-sample LSE:", LSE)
    
        if LSE < LSE_best:
            LSE_best = LSE
            SelectID_Best = SelectID
    
    

In [ ]:
TF_Select = list()
for idt in SelectID_Best:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
print(SelectID_Best)

TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
TF_Select = list()
for idt in SelectID_Best:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
print(SelectID_Best)

TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
uout, obj, _ = minConF_PQN(funObj_PPI, uSimplex, funProj_GL_LB, options)

In [ ]:
uout_ppi, zout_ppi = ProjCSimplexGL_Gurobi_LB_v2(uout, k, Group, h)
print(zout_ppi.flatten())
SelectID = Extract_Solution(uout_ppi, zout_ppi, Group, k, h)


TF_Select = list()
for idt in SelectID:
    TF_Select.append(TFList_in_Data[idt])
print(TF_Select)
print(SelectID)

In [ ]:
TF_Group = Show_TFs_in_Data(TG, pbmc_3k_scATAC_Gene_Enhancer_CD4_psum, ColSelect, Map_Gene_GTOpenR, TF_Select)

In [ ]:
ut = np.zeros((NumFeature,1))
ut[SelectID,0] = 1.0

ObjF, _  = L0GL_PPI_Obj(ut, X, y, S, pho, lamb)
print("Obj:", ObjF)
print("Short Dis;", ut.T@S@ut)


XSelect = X[:,SelectID]
w_est = np.linalg.pinv(XSelect.T @ XSelect) @ XSelect.T @ y
print(TF_Select)
print(np.abs(w_est.flatten()))

X_outsample_select = X_outsample[:, SelectID]
LSE = np.linalg.norm(y_outsample - X_outsample_select @ w_est)
print("Out-of-sample LSE:", LSE)

In [ ]:
plt.plot(y_outsample)
plt.plot(X_outsample_select @ w_est)

In [ ]:
uout_ppi = ProjCSimplexGL_Gurobi_LB(uout, k, Group, h)

In [ ]:
np.argsort(-zout_ppi.flatten())

In [ ]:
zout_ppi.flatten()

In [ ]:
Ind = np.argsort(-uout_ppi.flatten())[0:30]
print(Ind)
TFRes = list()
for idt in Ind:
    #print(TFList_in_Data[idt])
    TFRes.append(TFList_in_Data[idt])
print(TFRes)

In [ ]:
TFList_Raw = Tmp_Input['TF_Matching_p-value_1e-05'].tolist()
Chr_Position_Raw = Tmp_Input['chr_position'].tolist()
count = 0
for tfl, chp in zip(TFList_Raw, Chr_Position_Raw):
    if chp in Map_Gene_GTOpenR[GL[0]]:
        hit = 1
    else:
        hit = 0
    olp = list(set(tfl) & set(TFRes))
    print(count, olp, hit)
    count = count + 1

In [ ]:
uout.T @ S @ uout

In [ ]:
ShortPath = ShorestPath_GivenTFs(G_PPI_BioGrid, TFRes, "TestShortPath_Result.txt")
print(ShortPath)

# Tmp

In [ ]:
Tmp_Input

In [ ]:
Tmp_GT

In [ ]:
Tmp_Input

In [ ]:
tsp

In [ ]:
TFList_Raw = Tmp['TFs_binding'].tolist()
Hit_Raw = Tmp['Hit'].tolist()
for tfl,hit in zip(TFList_Raw, Hit_Raw):
    olp = list(set(tfl) & set(Gene_scRNA))
    print(olp, hit)

In [ ]:
Tmp['baitNameList'][0]

In [ ]:
for gt in Gene_Validate:
    GL = [gt]
    #print(GL)
    Tmp = CD4_scATAC_Gene_TF_Enhancer_PCHiC_Trim[CD4_scATAC_Gene_TF_Enhancer_PCHiC_Trim['Genes_within_250k'].apply(lambda x: any(gene in x for gene in GL))].reset_index()

    TF_Candiates =  list(set(list(itertools.chain.from_iterable(Tmp['TFs_binding'].tolist()))))
    #print(TF_Candiates)

    TF_In_Data = list(set(Gene_scRNA) & set(TF_Candiates))
    #print(TF_In_Data)
    
    Num_GT = Tmp[~Tmp['baitNameList'].isnull()]['baitNameList'].apply(lambda x: any(gene in x for gene in GL)).sum()
    
    print(gt, "Total_TF / TF_in_Data = ", len(TF_Candiates), "/", len(TF_In_Data), "GT number:", Num_GT)
    

In [ ]:
CD4_scATAC_Gene_TF_Enhancer[['chr', 'start', 'end']] = CD4_scATAC_Gene_TF_Enhancer['chr_position'].str.split('_', expand=True)

In [ ]:
CD4_scATAC_Gene_TF_Enhancer

In [ ]:

GL = [Gene_Validate[0]]
    #print(tg)
Tmp = CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['output_grch38_col4'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
TG_TFList = list(set(list(itertools.chain.from_iterable(Tmp['output_fimo_col4'].tolist()))))
    #print("TFs for the target is", len(TG_TFList))

TG_TGList_InData = list(set(TG_TFList) & set(Gene_scRNA))
    #print("TFs for the target in the data is", len(TG_TGList_InData))


In [ ]:
Tmp

In [ ]:
GeneCount = 0
TruthRegion = 0
Gene_with_GroundTruth = list()
True_OpenRegin_List = list()
for tg in Gene_Validate:
    GL = [tg]
    Tmp = CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['output_grch38_col4'].apply(lambda x: any(gene in x for gene in GL))].reset_index()
    TG_TFList = list(set(list(itertools.chain.from_iterable(Tmp['output_fimo_col4'].tolist()))))
    #print("TFs for the target is", len(TG_TFList))

    TG_TGList_InData = list(set(TG_TFList) & set(Gene_scRNA))
    #print("TFs for the target in the data is", len(TG_TGList_InData))

    Tmp = Tmp[Tmp['sum']>=1]
    #print(Tmp)

    CD4_Target = CD4_PCHiC[CD4_PCHiC['baitNameList'].apply(lambda lst: tg in lst) ]

    Start_val = Tmp['start'].values.astype(int)
    End_val = Tmp['end'].values.astype(int)

    oeStart_val = CD4_Target['oeStart'].values
    oeEnd_val = CD4_Target['oeEnd'].values

    start_mask = np.logical_and(
        Start_val[:, np.newaxis] >= oeStart_val-500,
        Start_val[:, np.newaxis] <= oeEnd_val+500
    )

    end_mask = np.logical_and(
        End_val[:, np.newaxis] >= oeStart_val-500,
        End_val[:, np.newaxis] <= oeEnd_val+500
    )


    HitList=np.array(np.logical_or(start_mask, end_mask).sum(axis=1))
    #print(HitList)
    Ind = np.where(HitList)[0]
    #print(Ind)
    #print(Tmp['chr_position'])
    #print(Tmp['chr_position'][Ind].values)
    OP_Hit_List = Tmp['chr_position'][Ind].values
    #print(OP_Hit_List)
    
    Overlap_regions = np.logical_or(start_mask, end_mask).any(axis=1).sum()
    if Overlap_regions > 0:
        GeneCount = GeneCount + 1
        TruthRegion = TruthRegion + Overlap_regions
        Gene_with_GroundTruth.append(tg)
        #print(tg)
        #print("TFs for the target is", len(TG_TFList))
        #print("TFs for the target in the data is", len(TG_TGList_InData))
        print(tg, "open reigions(", Tmp.shape[0], ")overlaps with", Overlap_regions, "within the PCHiC", CD4_Target.shape[0], "TF_Raw/TF_Exp=", len(TG_TFList), "/", len(TG_TGList_InData))
        True_OpenRegin_List.extend(OP_Hit_List)
        #print(OP_Hit_List[0])
        

In [ ]:
print("Gene:", len(Gene_with_GroundTruth), "Links", TruthRegion)

In [ ]:
len(set(True_OpenRegin_List))

In [ ]:
CD4_scATAC_Gene_TF_Enhancer['Hit_PCHiC'] = 0

In [ ]:
CD4_scATAC_Gene_TF_Enhancer['Hit_PCHiC'] = CD4_scATAC_Gene_TF_Enhancer['chr_position'].apply(lambda x: 1 if x in True_OpenRegin_List else 0)

In [ ]:
CD4_scATAC_Gene_TF_Enhancer['chr_position'].apply(lambda chrp: chrp in True_OpenRegin_List)['Hit_PCHiC'] = 1

In [ ]:
CD4_scATAC_Gene_TF_Enhancer

In [ ]:
CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['chr_position'] == True_OpenRegin_List[0]]

In [ ]:
Tmp = CD4_scATAC_Gene_TF_Enhancer[CD4_scATAC_Gene_TF_Enhancer['output_grch38_col4'].apply(lambda x: any(gene in x for gene in ['MAPKAPK2']))].reset_index()


In [ ]:
Tmp

In [ ]:
Tmp_TFs = Tmp['output_fimo_col4'].tolist()
for tfl in Tmp_TFs:
    Int = list(set(TG_TGList_InData) & set(tfl))
    print(sorted(Int))

In [ ]:
sp = nx.shortest_path_length(G_PPI_BioGrid, 'PAX5', 'SP2')
print(sp)
Path = nx.shortest_path(G_PPI_BioGrid, source='PAX5', target='SP2')
print(Path)

In [ ]:
Tmp

In [ ]:
Tmp[['chr', 'start', 'end']] = Tmp['chr_position'].str.split('_', expand=True)

In [ ]:
Tmp

In [ ]:
CD4_Target = CD4_PCHiC[CD4_PCHiC['baitNameList'].apply(lambda lst: TargetG in lst) ]

In [ ]:
Start_val = Tmp['start'].values.astype(int)
End_val = Tmp['end'].values.astype(int)

oeStart_val = CD4_Target['oeStart'].values
oeEnd_val = CD4_Target['oeEnd'].values

In [ ]:
start_mask = np.logical_and(
        Start_val[:, np.newaxis] >= oeStart_val,
        Start_val[:, np.newaxis] <= oeEnd_val
    )

end_mask = np.logical_and(
        End_val[:, np.newaxis] >= oeStart_val,
        End_val[:, np.newaxis] <= oeEnd_val
    )

Overlap_regions = np.logical_or(start_mask, end_mask).any(axis=1).sum()

In [ ]:
Overlap_regions

In [ ]:
CD4_Target

In [ ]:
int(gene_tss[gene_tss['genes'] == TargetG]['tss_corrected'].values[0])+250000, int(gene_tss[gene_tss['genes'] == TargetG]['tss_corrected'].values[0])-250000

In [ ]:
print(Tmp['output_fimo_col4'].tolist())

In [ ]:
GL